# **7장: 파인튜닝**

파인튜닝은 모델의 전체 또는 일부를 추가적으로 학습시켜 특정 작업에 맞게 모델을 조정하는 과정입니다. 5장과 6장에서는 프롬프트 기반 방법에 대해 다루며, 이는 모델에 지침, 맥락, 도구를 제공하여 모델을 조정하는 방법입니다. 반면, 파인튜닝은 모델의 가중치를 조정하여 모델을 적응시킵니다.

파인튜닝은 모델의 여러 측면을 향상시킬 수 있습니다. 이를 통해 코딩이나 의료 질문 응답과 같은 특정 도메인 기능을 향상시키거나 모델의 안전성을 강화할 수 있습니다. 그러나 주로 모델의 지시 사항 준수 능력을 개선하는 데 사용되며, 특히 특정 출력 스타일과 형식을 준수하도록 조정하는 데 유용합니다.

파인튜닝은 필요에 맞춘 모델을 만들 수 있도록 도와주지만, 더 많은 초기 투자가 필요합니다. "언제 파인튜닝을 해야 하고, 언제 RAG(검색 기반 생성)을 해야 하는가?"라는 질문을 자주 듣습니다. 이 장에서는 파인튜닝 개요와 함께 파인튜닝을 해야 하는 이유와 하지 말아야 하는 이유를 논의하고, 파인튜닝과 다른 방법을 선택하는 데 있어 간단한 프레임워크를 제시합니다.

프롬프트 기반 방법과 비교했을 때, 파인튜닝은 더 높은 메모리 사용량을 요구합니다. 현재의 기초 모델 규모에서 단순 파인튜닝은 종종 단일 GPU에서 처리할 수 있는 메모리를 초과합니다. 이로 인해 파인튜닝은 비용이 많이 들고 도전 과제가 됩니다. 이 장에서는 메모리 사용량 감소가 파인튜닝 기술의 주요 동기가 되는 이유와, 이를 이해하는 데 중요한 기술적 요인에 대해 논의합니다.

파인튜닝 분야에서 점점 더 지배적인 접근법은 PEFT(매개변수 효율적 파인튜닝)입니다. 이 장에서는 PEFT와 기존 파인튜닝 방식의 차이점, 그리고 이 기술의 발전 과정을 살펴봅니다. 특히 어댑터 기반 기술이라는 설득력 있는 범주에 중점을 둡니다.

프롬프트 기반 방법에서는 ML 모델의 동작 원리를 깊이 이해하지 않아도 사용할 수 있습니다. 그러나 파인튜닝은 모델 훈련의 영역으로 들어가는 것이므로 ML 기본 지식이 필요합니다. ML 기본 개념은 이 책의 범위를 벗어나지만, 책의 GitHub 저장소에서 유용한 자료를 참고할 수 있습니다. 이 장에서는 논의와 직접적으로 관련된 몇 가지 핵심 개념을 다룹니다.

이 장은 제가 작성하기에 가장 기술적으로 어려운 장입니다. 이는 개념의 복잡성 때문이 아니라, 이 개념들이 포괄적으로 다루는 범위 때문입니다. 읽는 입장에서도 기술적으로 어려울 수 있습니다. 만약 너무 세부적인 내용을 다룬다고 느껴진다면 필요한 부분만 선택적으로 읽어도 괜찮습니다.

논의할 내용이 많습니다. 시작해봅시다!

---

## **파인튜닝 개요**

파인튜닝을 위해서는 기본 모델이 필요합니다. 이 기본 모델은 여러분이 필요로 하는 모든 기능을 갖추고 있지는 않지만, 특정 작업에 충분히 잘 작동하도록 조정할 수 있습니다.

파인튜닝은 전이 학습의 한 방식입니다. 전이 학습은 1976년 Bozinovski와 Fulgosi가 처음 제안한 개념으로, 한 작업에서 얻은 지식을 새로운 관련 작업 학습에 활용하는 방법을 연구합니다. 예를 들어, 피아노 연주법을 아는 것이 새로운 악기를 배우는 데 도움이 되는 것과 유사합니다.

전이 학습의 초기 대규모 성공 사례는 Google의 다국어 번역 시스템(Johnson et al., 2016)이었습니다. 이 모델은 포르투갈어-영어와 영어-스페인어 번역에 대한 지식을 활용해 포르투갈어-스페인어 번역을 수행했습니다. 포르투갈어-스페인어 번역 데이터가 없었음에도 말입니다.

딥러닝 초기부터 전이 학습은 제한적이거나 비용이 많이 드는 훈련 데이터를 필요로 하는 작업에 솔루션을 제공했습니다. 방대한 데이터를 가진 작업에 기본 모델을 학습시키고, 이를 목표 작업으로 전이할 수 있습니다.

LLMs(대규모 언어 모델)에서는 텍스트 완성(방대한 데이터가 있는 작업)에서 얻은 지식을 법률 질문 응답이나 텍스트-데이터베이스 질의(SQL)와 같은 더 전문화된 작업으로 전이할 수 있습니다. 이는 기초 모델의 가치를 더욱 높이는 중요한 기능입니다.

전이 학습은 샘플 효율성을 향상시켜 적은 예제 수로도 동일한 동작을 학습할 수 있도록 합니다. 예를 들어, 법률 질문 응답 모델을 처음부터 훈련시키려면 수백만 개의 예제가 필요할 수 있지만, 좋은 기본 모델을 파인튜닝하는 데는 수백 개의 예제만 필요할 수 있습니다.

이상적으로는, 모델이 학습해야 할 대부분의 내용은 이미 기본 모델에 존재하며, 파인튜닝은 모델의 동작을 세부적으로 조정하는 데 사용됩니다. OpenAI의 InstructGPT 논문(2022)은 파인튜닝을 사용자가 프롬프트만으로는 접근하기 어려운 모델의 잠재적 기능을 "잠금 해제"하는 과정으로 제안했습니다.

---

>**참고**
>
>파인튜닝이 전이 학습을 하는 유일한 방법은 아닙니다. 또 다른 접근법은 **특징 기반 전이(feature-based transfer)**입니다. 이 방법에서는 데이터를 기반으로 특징(일반적으로 임베딩 벡터)을 추출하도록 모델을 훈련시키고, 이 특징을 다른 모델에서 사용합니다. 특징 기반 전이는 이미지넷 데이터셋을 활용한 이미지 객체 탐지나 분할 작업 등 컴퓨터 비전 분야에서 특히 흔히 사용됩니다.

---

파인튜닝은 모델 훈련 과정의 일부이며, 모델 사전 학습(pre-training)의 확장입니다. 사전 학습 이후에 이루어지는 모든 학습은 파인튜닝으로 간주되며, 다양한 형태를 취할 수 있습니다. 2장에서 이미 감독형 파인튜닝(supervised finetuning)과 선호도 파인튜닝(preference finetuning)의 두 가지 유형을 다뤘습니다. 이 방법들을 간략히 요약하고 애플리케이션 개발자로서 이를 어떻게 활용할 수 있는지 살펴보겠습니다.

모델의 훈련 과정은 일반적으로 자기지도 학습(self-supervision)을 통해 이루어지는 사전 학습에서 시작됩니다. 자기지도 학습은 모델이 대량의 라벨 없는 데이터로부터 학습할 수 있도록 합니다. 언어 모델의 경우, 자기지도 데이터는 일반적으로 주석이 필요 없는 단순한 텍스트 시퀀스입니다.

비싼 작업별 데이터로 사전 학습된 모델을 파인튜닝하기 전에, 저렴한 작업 관련 데이터를 사용해 자기지도 학습으로 파인튜닝할 수 있습니다. 예를 들어, 법률 질문 응답 모델을 파인튜닝하려면, 비싼 주석이 달린 데이터(질문-답변) 대신 먼저 원문 법률 문서로 모델을 파인튜닝할 수 있습니다. 마찬가지로, 베트남어로 된 책 요약 모델을 만들려면, 먼저 방대한 베트남어 텍스트 컬렉션으로 모델을 파인튜닝할 수 있습니다. 이러한 자기지도 파인튜닝은 "지속적 사전 학습(continued pre-training)"이라고도 불립니다.

1장에서 논의했듯이, 언어 모델은 오토리그레시브(autoregressive) 모델이거나 마스크드(masked) 모델일 수 있습니다. 오토리그레시브 모델은 이전 토큰을 맥락으로 사용해 다음 토큰을 예측합니다. 마스크드 모델은 앞뒤 토큰을 사용해 빈칸을 채웁니다. 마찬가지로, 감독형 파인튜닝을 사용하면 모델이 다음 토큰을 예측하거나 빈칸을 채우도록 학습시킬 수 있습니다. 후자는 "인필링 파인튜닝(infilling finetuning)"이라고도 하며, 텍스트 편집이나 코드 디버깅 같은 작업에 특히 유용합니다. 오토리그레시브 방식으로 사전 학습된 경우에도 인필링 작업을 위해 모델을 파인튜닝할 수 있습니다.

자기지도 학습 중 모델이 학습할 수 있는 방대한 데이터는 모델이 세상을 깊이 이해하도록 돕습니다. 하지만 사용자가 자신의 작업에 필요한 지식을 모델에서 추출하거나, 모델의 동작이 인간의 선호와 맞지 않을 수 있습니다. 감독형 파인튜닝은 고품질 주석 데이터를 사용하여 모델의 동작을 인간의 사용 및 선호에 맞게 정제합니다.

감독형 파인튜닝 동안, 모델은 (입력, 출력) 쌍을 사용해 학습됩니다. 입력은 명령어가 될 수 있고 출력은 응답이 될 수 있습니다. 응답은 책 요약 작업처럼 개방형일 수도 있고, 분류 작업처럼 폐쇄형일 수도 있습니다. 특히 사실적 일관성, 도메인 전문 지식, 또는 정치적 올바름이 요구되는 지침의 경우, 고품질 지침 데이터는 생성하기 어렵고 비용이 많이 들 수 있습니다. 8장에서는 이러한 지침 데이터를 획득하는 방법을 다룹니다.

또한 모델은 강화 학습을 통해 인간 선호도를 극대화하는 응답을 생성하도록 파인튜닝될 수 있습니다. 선호도 파인튜닝은 일반적으로 (명령어, 우승 응답, 패배 응답) 형식을 따르는 비교 데이터를 필요로 합니다.

모델의 컨텍스트 길이를 확장하기 위해 파인튜닝하는 것도 가능합니다. "긴 컨텍스트 파인튜닝(Long-context finetuning)"은 일반적으로 모델의 아키텍처를 수정하는 것을 포함하며, 예를 들어 위치 임베딩(positional embeddings)을 조정합니다. 긴 시퀀스는 더 많은 토큰 위치를 의미하며, 위치 임베딩은 이를 처리할 수 있어야 합니다. 다른 파인튜닝 기술과 비교할 때, 긴 컨텍스트 파인튜닝은 더 어렵습니다. 결과적으로, 모델은 짧은 시퀀스에서 성능이 저하될 수도 있습니다.

**그림 7-1**은 Llama 2 기반 모델(Code Llama 모델들)의 제작 과정을 보여줍니다. 긴 컨텍스트 파인튜닝을 사용해, 모델의 최대 컨텍스트 길이를 4,096 토큰에서 16,384 토큰으로 늘려 더 긴 코드 파일을 처리할 수 있었습니다. 그림에서, "지시 기반 파인튜닝"은 감독형 파인튜닝을 의미합니다.

파인튜닝은 모델 개발자와 애플리케이션 개발자 모두 수행할 수 있습니다. 모델 개발자는 일반적으로 배포 전에 여러 파인튜닝 기술로 모델을 후속 훈련합니다. 또한 애플리케이션 개발자가 가장 적합한 버전을 선택할 수 있도록, 서로 다른 정도로 파인튜닝된 다양한 모델 버전을 출시할 수도 있습니다.

<img src="./images/fig_07_01.png" width="800">

애플리케이션 개발자로서 사전 학습된 모델을 파인튜닝할 수도 있지만, 대부분의 경우 후속 학습(post-trained)된 모델을 파인튜닝하게 될 가능성이 높습니다. 모델이 더 세밀하게 정제되고, 작업에 더 적합한 지식을 가지고 있을수록, 이를 작업에 맞게 조정하는 데 필요한 작업량이 줄어듭니다.
---

## **파인튜닝을 할 때**  

다양한 파인튜닝 기법에 뛰어들기 전에, 파인튜닝이 올바른 선택인지 고려하는 것이 중요합니다. 프롬프트 기반 방법과 비교했을 때, 파인튜닝은 데이터와 하드웨어뿐만 아니라 ML 전문 인력에 있어 훨씬 더 많은 자원을 필요로 합니다. 따라서 일반적으로 파인튜닝은 프롬프트 기반 방법으로 광범위한 실험을 수행한 후 시도됩니다. 하지만 파인튜닝과 프롬프트 사용은 상호 배타적인 것이 아닙니다. 실제 문제는 종종 두 가지 접근 방식을 모두 요구합니다.

### **파인튜닝의 주요 이유**  

파인튜닝의 주요 목적은 모델의 품질을 향상시키는 데 있으며, 이는 일반적인 기능과 작업별 기능 모두에 해당됩니다. 파인튜닝은 JSON이나 YAML 형식과 같은 특정 구조를 따르는 출력 생성을 개선하기 위해 자주 사용됩니다.

다목적 모델은 다양한 벤치마크에서 우수한 성능을 보일 수 있지만, 특정 작업에서는 잘 작동하지 않을 수 있습니다. 만약 원하는 모델이 작업에 충분히 학습되지 않았다면, 해당 데이터를 사용하여 모델을 파인튜닝하는 것이 특히 유용할 수 있습니다.

예를 들어, 기본 모델은 일반적인 SQL 방언 변환에는 적합할 수 있지만, 드문 SQL 방언에서는 실패할 수 있습니다. 이런 경우, 해당 SQL 방언을 포함하는 데이터로 모델을 파인튜닝하면 도움이 됩니다. 마찬가지로, 모델이 일반적인 SQL 쿼리에는 적합하지만, 고객별 쿼리에는 종종 실패한다면, 고객별 쿼리에 대해 모델을 파인튜닝하는 것이 유용할 수 있습니다.

파인튜닝의 또 다른 흥미로운 사례는 **편향 완화(bias mitigation)**입니다. 기본 모델이 학습 데이터에서 특정 편향을 유지한다면, 파인튜닝 시 신중히 선택된 데이터를 사용해 이러한 편향을 완화할 수 있습니다(Wang & Russakovsky, 2023). 예를 들어, 모델이 CEO 이름에 지속적으로 남성적인 이름을 할당한다면, 여성 CEO 이름이 포함된 데이터셋으로 파인튜닝하면 이 편향을 완화할 수 있습니다. Garimella 등(2022)은 여성 저자가 작성한 텍스트로 BERT 모델을 파인튜닝하면 성별 편향을 줄이고, 아프리카 저자가 작성한 텍스트로 파인튜닝하면 인종 편향을 줄일 수 있음을 발견했습니다.

큰 모델을 더 개선하기 위해 파인튜닝할 수도 있지만, **작은 모델을 파인튜닝하는 경우가 훨씬 더 일반적**입니다. 작은 모델은 더 적은 메모리를 요구하므로 파인튜닝이 더 쉽고, 생산 환경에서 사용하기에도 더 저렴하고 빠릅니다.

일반적인 접근법 중 하나는 **지식 증류(distillation)**입니다. 이는 큰 모델이 생성한 데이터를 사용해 작은 모델이 큰 모델의 동작을 모방하도록 파인튜닝하는 것입니다. 이 방법은 큰 모델의 지식을 작은 모델로 증류하는 방식이며, 8장에서 다른 데이터 합성 기법과 함께 논의됩니다.

작은 모델이 특정 작업에서 대형 모델을 능가할 수도 있습니다. 예를 들어, Grammarly는 Flan-T5S 모델(Chung et al., 2022)을 파인튜닝하여, 텍스트 편집 작업에서 GPT-3 변형 모델보다 뛰어난 성능을 보였으며, 이는 60배 더 작은 모델임에도 불구하고 이루어진 결과입니다. 이 파인튜닝은 약 82,000개의 (명령, 출력) 쌍 데이터를 사용했으며, 이는 처음부터 텍스트 편집 모델을 훈련시키는 데 필요한 데이터 양보다 적습니다.

---

### **파인튜닝을 하지 말아야 할 이유**  
파인튜닝은 모델의 성능을 다양한 방식으로 향상시킬 수 있지만, 이러한 개선 중 많은 부분은 파인튜닝 없이도 어느 정도 달성할 수 있습니다. 잘 설계된 프롬프트와 컨텍스트를 사용하면 모델의 성능을 개선할 수 있습니다. 또한 파인튜닝은 구조화된 출력 생성을 돕지만, 2장에서 논의된 다른 기술들도 이와 동일한 역할을 할 수 있습니다.

첫째, 특정 작업을 위해 모델을 파인튜닝하면 해당 작업에 대한 성능은 개선되지만, 다른 작업에 대한 성능은 저하될 수 있습니다. 이는 다양한 프롬프트를 기대하는 애플리케이션에서 실망스러울 수 있습니다.

둘째, 프로젝트를 처음 시작할 때는 파인튜닝이 가장 먼저 시도해야 할 일이 아닙니다. 파인튜닝은 높은 초기 투자와 지속적인 유지보수가 필요합니다. 우선, 데이터가 필요합니다. 주석이 달린 데이터는 특히 비판적 사고와 도메인 전문 지식을 요구하는 작업에서 수집 속도가 느리고 비용이 많이 들 수 있습니다.

셋째, 파인튜닝된 모델을 사용하려면 이를 제공하는 방법도 결정해야 합니다. 예를 들어, 직접 호스팅할지, API 서비스를 사용할지 결정해야 하며, 대규모 모델(LMMs)에서의 추론 최적화는 간단한 일이 아닙니다. 

마지막으로, 파인튜닝 대신 새 모델로 전환하는 것이 더 적은 향상만 가져올 경우 프로젝트가 우선순위에서 밀릴 수 있습니다.  

AI 실험은 **6장에서 논의된 모범 사례를 따르며 프롬프트(Prompting)**에서 시작해야 합니다. 프롬프트만으로 충분하지 않은 경우에만 더 고급 솔루션을 탐색하십시오. 다양한 프롬프트를 철저히 테스트했는지 확인해야 하며, 모델 성능은 프롬프트에 따라 크게 달라질 수 있습니다.

많은 실무자들은 다음과 같은 이야기를 공유합니다. 누군가 프롬프트가 효과적이지 않다고 불평하며 파인튜닝을 고집합니다. 그러나 조사해 보면, 프롬프트 실험이 최소화되었거나 체계적이지 않았음을 알 수 있습니다. 지침이 불분명하거나, 예제가 실제 데이터를 나타내지 못했으며, 메트릭도 제대로 정의되지 않은 경우가 많습니다. 프롬프트 실험 과정을 정제한 후에는 프롬프트 품질이 충분히 향상되어 애플리케이션에 적합한 결과를 얻었습니다.

---

**도메인별 작업의 파인튜닝**  
"범용 모델은 도메인별 작업에서 잘 작동하지 않는다"라는 주장에 주의하십시오. 따라서 특정 작업을 위해 모델을 파인튜닝하거나 새로 훈련해야 한다는 의견이 있을 수 있습니다. 하지만 범용 모델이 점점 더 유능해지면서 도메인별 작업에서도 더 나은 성능을 발휘할 수 있으며, 도메인별 모델을 능가할 수 있습니다.

흥미로운 초기 사례 중 하나는 **BloombergGPT**입니다. 이 모델은 2023년 3월 Bloomberg에서 도입되었으며, 금융 작업에서 우수한 성능을 발휘하고 민감한 데이터를 다루기 위해 내부에서 호스팅할 수 있는 중간 크기의 모델이었습니다. 500억 개의 매개변수를 가진 이 모델은 훈련에 130만 A100 GPU 시간을 필요로 했으며, 계산 비용은 데이터 비용을 제외하고 약 130만 달러에서 260만 달러로 추정됩니다(Wu et al., 2023).

같은 달, OpenAI는 **GPT-4-0314**를 출시했으며, Li et al. (2023)의 연구에 따르면 GPT-4-0314는 다양한 금융 벤치마크에서 BloombergGPT를 크게 능가했습니다. **표 7-1**은 두 가지 벤치마크의 세부 정보를 제공합니다.

| 모델                  | FiQA 감정 분석 (가중 F1) | ConvFinQA (정확도) |
|-----------------------|------------------------|-------------------|
| GPT-4-0314 (제로 샷) | 87.15                 | 76.48            |
| BloombergGPT          | 75.07                 | 43.41            |

이후, GPT-4에 견줄 만한 성능을 가진 중간 크기 모델이 여러 개 출시되었습니다. 여기에는 **Claude 3.5 Sonnet(70B 매개변수)**, **Llama 3-70B-Instruct**, **Qwen2-72B-Instruct** 등이 포함되며, 후자의 두 모델은 오픈 웨이트(Open Weight)로 제공되어 자체 호스팅이 가능합니다.

벤치마크만으로는 실제 세계의 성능을 충분히 측정할 수 없기 때문에 BloombergGPT는 특정 사용 사례에서 Bloomberg에 적합하게 작동할 가능성이 있습니다. Bloomberg 팀은 이 모델을 훈련하면서 얻은 경험을 통해 향후 더 나은 모델을 개발하고 운영할 수 있는 귀중한 통찰을 얻었습니다.

---

파인튜닝과 프롬프트 실험은 모두 체계적인 과정을 요구합니다. 프롬프트 실험을 수행하면 평가 파이프라인, 데이터 주석 지침, 그리고 실험 추적 관행을 구축할 수 있으며, 이는 파인튜닝의 중요한 초석이 됩니다.

프롬프트 캐싱(Prompt Caching)이 도입되기 전, 파인튜닝의 한 가지 이점은 토큰 사용을 최적화하는 데 도움이 된다는 점이었습니다. 프롬프트에 더 많은 예제를 추가할수록 모델이 사용하는 입력 토큰 수가 증가해 지연 시간(latency)과 비용도 증가합니다. 각 프롬프트에 예제를 포함하는 대신, 해당 예제들로 모델을 파인튜닝하면 짧은 프롬프트로 동일한 결과를 얻을 수 있습니다(그림 7-2 참고).

<img src="./images/fig_07_02.png" width="800">

프롬프트 캐싱은 반복적인 프롬프트 세그먼트를 재사용할 수 있도록 캐싱하는 방식으로, 이러한 이점을 줄일 수 있습니다. 프롬프트 캐싱에 대한 논의는 **9장에서** 다뤄집니다. 그러나 프롬프트에서 사용할 수 있는 예제 수는 여전히 최대 컨텍스트 길이에 의해 제한됩니다. 반면, 파인튜닝에서는 사용할 수 있는 예제 수에 제한이 없습니다.

---

### **파인튜닝과 RAG**  

프롬프트를 통해 성능을 최대한 끌어올린 후, RAG(검색 기반 생성) 또는 파인튜닝 중 어떤 것을 선택해야 할지 고민할 수 있습니다. 이 선택은 모델의 실패 원인이 정보 기반인지 행동 기반인지에 따라 달라집니다.

**모델이 정보 부족으로 실패하는 경우**, RAG 시스템을 사용하여 모델이 관련 정보 소스에 접근하도록 하면 도움이 될 수 있습니다. 정보 기반 실패는 출력이 사실적으로 틀리거나 오래된 경우에 발생합니다. 다음은 정보 기반 실패가 발생하는 두 가지 예시입니다:

1. **모델에 정보가 없는 경우**  
   공용 모델은 사용자의 조직 또는 개인적인 정보에 대한 접근 권한이 없을 가능성이 높습니다. 모델이 필요한 정보를 가지고 있지 않을 때, 해당 정보를 가지고 있지 않다는 응답을 하거나 가짜로 응답을 생성할 수 있습니다.

2. **모델의 정보가 오래된 경우**  
   예를 들어, "Taylor Swift가 발매한 스튜디오 앨범의 개수는 몇 개인가요?"라고 질문했을 때, 정답이 11개인데 모델이 10개라고 답한다면, 이는 모델의 데이터 컷오프 날짜가 최신 앨범 발매 이전일 수 있기 때문입니다.

Ovadia 등(2024)의 논문 **"Fine-Tuning or Retrieval?"**에 따르면, 최신 정보가 필요한 작업(예: 현재 이벤트에 관한 질문)에서는 RAG가 파인튜닝된 모델보다 우수한 성능을 보였습니다. 뿐만 아니라, 기본 모델에 RAG를 적용한 경우가 파인튜닝된 모델에 RAG를 적용한 경우보다 더 나은 성능을 보였습니다(표 7-2 참조). 이는 **파인튜닝이 특정 작업에서 모델의 성능을 향상시킬 수 있지만, 다른 영역에서는 성능 저하로 이어질 수도 있다**는 점을 시사합니다.

**표 7-2. RAG는 최신 이벤트에 대한 질문-응답 작업에서 파인튜닝보다 뛰어난 성능을 발휘**  
Ovadia et al.(2024)이 큐레이션한 결과에 따르면, FT-reg와 FT-par는 두 가지 서로 다른 파인튜닝 접근법을 나타냅니다.

| 모델        | 기본 모델 | 기본 모델 + RAG | FT-reg | FT-par | FT-reg + RAG | FT-par + RAG |
|-------------|-----------|----------------|--------|--------|--------------|--------------|
| Mistral-7B  | 0.481     | 0.875          | 0.504  | 0.588  | 0.810        | 0.830        |
| Llama 2-7B  | 0.353     | 0.585          | 0.219  | 0.392  | 0.326        | 0.520        |
| Orca 2-7B   | 0.456     | 0.876          | 0.511  | 0.566  | 0.820        | 0.826        |

모델에 행동적 문제가 있는 경우, 파인튜닝이 도움이 될 수 있습니다. 행동적 문제란 모델의 출력이 사실적으로 정확하지만 작업과 무관한 경우를 말합니다. 예를 들어, 소프트웨어 프로젝트에 대한 기술 사양을 생성하도록 모델에 요청했을 때, 출력된 사양이 정확하지만 엔지니어 팀이 필요로 하는 세부 정보를 포함하지 않는 경우입니다. 이때, 잘 정의된 기술 사양 데이터를 사용해 모델을 파인튜닝하면 출력이 더 관련성 있게 될 수 있습니다.

또 다른 문제는 예상된 출력 형식을 따르지 못하는 경우입니다. 예를 들어, 모델에 HTML 코드를 작성하도록 요청했지만 생성된 코드가 컴파일되지 않는 경우, 이는 모델이 학습 데이터에서 HTML에 충분히 노출되지 않았기 때문일 수 있습니다. 파인튜닝을 통해 더 많은 HTML 코드를 학습시키면 이를 해결할 수 있습니다.

**의미 구문 분석(semantic parsing)**은 모델이 예상된 형식으로 출력을 생성할 수 있는 능력에 따라 성공 여부가 결정되는 작업의 한 범주로, 종종 파인튜닝이 필요합니다. 의미 구문 분석이란 자연어를 JSON과 같은 구조화된 형식으로 변환하는 것을 의미합니다. JSON, YAML, 정규식과 같은 간단한 구문에 대해서는 기존 모델이 일반적으로 잘 작동하지만, 인기가 적은 도구나 복잡한 구문에 대해서는 성능이 떨어질 수 있습니다.

**요약하자면, 파인튜닝은 "형식(form)"을 위한 것이고, RAG는 "사실(facts)"을 위한 것입니다.** RAG 시스템은 외부 지식을 제공하여 모델이 더 정확하고 유익한 답변을 생성하도록 돕습니다. RAG 시스템은 모델의 헛소리(hallucinations)를 완화할 수 있습니다. 반면, 파인튜닝은 모델이 구문과 스타일을 이해하고 따를 수 있도록 돕습니다. 고품질 데이터로 수행되는 경우 파인튜닝은 헛소리를 줄일 수 있지만, 데이터 품질이 낮으면 오히려 헛소리를 악화시킬 수 있습니다.

모델에 정보와 행동 문제가 모두 있는 경우, RAG부터 시작하세요. RAG는 일반적으로 훈련 데이터를 준비하거나 파인튜닝된 모델을 호스팅할 필요가 없기 때문에 더 간단합니다. RAG를 사용할 때는 벡터 데이터베이스를 바로 사용하는 대신, BM25와 같은 간단한 용어 기반 솔루션부터 시작하십시오.

Ovadia et al.(2024)의 연구에 따르면, MMLU 벤치마크의 거의 모든 질문 범주에서 RAG는 Mistral-7B, Llama 2-7B, Orca 2-7B 모델을 대상으로 한 파인튜닝보다 성능이 뛰어났습니다.

RAG와 파인튜닝은 상호 배타적이지 않습니다. 때로는 두 가지를 함께 사용하여 애플리케이션 성능을 극대화할 수 있습니다. 동일한 실험에서 Ovadia et al.(2024)은 파인튜닝된 모델 위에 RAG를 추가하면 MMLU 벤치마크에서 43%의 성능 향상을 제공할 수 있음을 보여주었습니다. 하지만 같은 실험에서, RAG와 파인튜닝된 모델을 함께 사용하는 것이 단독 RAG보다 57%의 경우 성능이 개선되지 않았습니다.

모든 애플리케이션에 대한 보편적인 워크플로는 없습니다. 그림 7-3은 애플리케이션 개발 프로세스가 시간이 지남에 따라 따를 수 있는 몇 가지 경로를 보여줍니다. 이 화살표는 다음에 시도해야 할 단계를 나타냅니다. 이 그림은 OpenAI(2023)의 워크플로 예제에서 영감을 얻었습니다.

<img src="./images/fig_07_03.png" width="800">

모델을 작업에 적응시키는 워크플로는 다음과 같이 작동할 수 있습니다. 적응 단계에 들어가기 전에, 4장에서 논의된 대로 평가 기준을 정의하고 평가 파이프라인을 설계해야 합니다. 이 평가 파이프라인은 애플리케이션 개발 과정에서 진행 상황을 측정하는 데 사용할 것입니다. 평가는 초기 단계에서만 이루어지는 것이 아니라, 과정의 모든 단계에서 수행되어야 합니다.

1. **프롬프트만으로 작업을 수행하도록 모델을 설정해보세요.**  
   5장에서 다룬 프롬프트 엔지니어링 모범 사례를 사용하고, 프롬프트 버전을 체계적으로 관리하세요.

2. **프롬프트에 더 많은 예제를 추가하세요.**  
   사용 사례에 따라 필요한 예제 수는 1개에서 50개 사이일 수 있습니다.

3. **모델이 정보를 누락해서 자주 실패한다면, 관련 정보를 제공할 수 있는 데이터 소스와 연결하세요.**  
   RAG(검색 기반 생성)를 사용할 때는 기본 검색 방법(예: 용어 기반 검색)으로 시작하세요. 간단한 검색 방법만으로도 적절하고 정확한 지식을 추가하면 모델 성능을 개선할 수 있습니다.

4. **모델의 실패 유형에 따라 다음 단계 중 하나를 탐색하세요:**  
   a. 모델이 계속해서 정보 기반 실패를 겪는다면, 임베딩 기반 검색과 같은 더 고급 RAG 방법을 시도해보세요.  
   b. 모델이 여전히 행동 문제(예: 관련 없거나 잘못된 형식 또는 안전하지 않은 응답 생성)를 겪는다면, 파인튜닝을 선택하세요. 임베딩 기반 검색은 파이프라인에 추가 구성 요소를 도입하여 추론 복잡성을 증가시키는 반면, 파인튜닝은 모델 개발의 복잡성을 높이지만 추론에는 영향을 미치지 않습니다.

5. **RAG와 파인튜닝을 결합하여 더 큰 성능 향상을 도모하세요.**

파인튜닝과 다른 대안 기술의 장단점을 모두 고려한 후 모델을 파인튜닝하기로 결정했다면, 이 장의 나머지 내용이 도움이 될 것입니다. 먼저, 파인튜닝의 가장 큰 도전 과제인 **메모리 병목 현상(memory bottleneck)**에 대해 살펴보겠습니다.

---

## **메모리 병목 현상**  

**파인튜닝은 메모리 사용량이 많은 작업이므로, 많은 파인튜닝 기술이 메모리 풋프린트를 최소화하는 데 초점을 맞추고 있습니다.** 메모리 병목 현상이 발생하는 이유를 이해하는 것은 이러한 기술이 작동하는 이유와 방법을 이해하는 데 필요합니다. 이 이해는 나아가 자신에게 가장 적합한 파인튜닝 방법을 선택하는 데 도움을 줄 수 있습니다.

파인튜닝의 메모리 병목 현상을 설명하는 것 외에도, 이 섹션에서는 각 모델의 메모리 사용량을 계산하는 간단한 공식을 소개합니다. 이 계산은 모델을 서비스하거나 파인튜닝하기 위해 필요한 하드웨어를 추정하는 데 유용합니다.

메모리 계산에는 저수준 ML 및 컴퓨팅 개념의 분해가 필요하므로 이 섹션은 기술적으로 다소 복잡할 수 있습니다. 이러한 개념에 익숙하다면 이 섹션을 건너뛰어도 좋습니다.

---

**메모리 병목 현상 이해를 위한 핵심 요점**

이 섹션을 건너뛰기로 결정했다면, 다음 몇 가지 핵심 요점을 참고하세요. 아래 요점 중 낯선 내용이 있다면, 이 섹션의 개념들이 이를 설명하는 데 도움이 될 것입니다:

1. **기초 모델의 규모 때문에 메모리는 추론과 파인튜닝 모두에서 작업의 병목 현상이 됩니다.**  
   신경망이 훈련되는 방식 때문에, 파인튜닝에 필요한 메모리는 일반적으로 추론에 필요한 메모리보다 훨씬 높습니다.

2. **파인튜닝 중 모델의 메모리 풋프린트에 기여하는 주요 요소는 파라미터 수, 학습 가능한 파라미터 수, 수치 표현 방식입니다.**

3. **학습 가능한 파라미터 수가 많을수록 메모리 풋프린트가 커집니다.**  
   학습 가능한 파라미터 수를 줄이면 파인튜닝에 필요한 메모리를 줄일 수 있습니다. 학습 가능한 파라미터 수를 줄이는 것은 PEFT(매개변수 효율적 파인튜닝)의 주요 동기입니다.

4. **양자화(Quantization)**는 모델을 더 많은 비트로 표현된 형식에서 더 적은 비트로 표현된 형식으로 변환하는 것을 의미합니다.  
   양자화는 모델의 메모리 풋프린트를 줄이는 간단하고 효율적인 방법입니다. 예를 들어, 130억 개의 파라미터를 가진 모델에서 FP32를 사용하는 경우, 가중치 당 4바이트 또는 전체 가중치에 대해 52GB가 필요합니다. 각 값을 2바이트로 줄이면, 모델의 가중치에 필요한 메모리는 26GB로 줄어듭니다.

5. **추론은 일반적으로 가능한 한 적은 비트를 사용해 수행됩니다.**  
   예를 들어, 16비트, 8비트, 심지어 4비트를 사용할 수 있습니다.

6. **훈련은 수치적 정밀도에 더 민감하므로, 낮은 정밀도로 모델을 훈련하는 것이 더 어렵습니다.**  
   훈련은 일반적으로 혼합 정밀도에서 수행되며, 일부 작업은 높은 정밀도(e.g., 32비트)로 수행되고, 일부는 낮은 정밀도(e.g., 16비트 또는 8비트)로 수행됩니다.

---


### 역전파와 학습 가능한 파라미터
파인튜닝 중 모델의 메모리 풋프린트를 결정하는 주요 요인은 **학습 가능한 파라미터의 수**입니다. 학습 가능한 파라미터는 파인튜닝 중 업데이트될 수 있는 파라미터를 의미합니다. 사전 학습 중에는 모든 모델 파라미터가 업데이트됩니다. 추론 중에는 모델 파라미터가 업데이트되지 않습니다. 파인튜닝 중에는 일부 또는 모든 모델 파라미터가 업데이트될 수 있습니다. 변경되지 않고 유지되는 파라미터를 **고정된 파라미터(frozen parameters)**라고 합니다.

각 학습 가능한 파라미터에 필요한 메모리는 모델이 훈련되는 방식에 따라 결정됩니다. 현재 신경망은 일반적으로 **역전파(Backpropagation)** 메커니즘을 사용하여 훈련됩니다. 역전파를 사용할 때, 각 훈련 단계는 두 단계로 구성됩니다:

1. **전방 패스(Forward pass):** 입력에서 출력을 계산하는 과정입니다.
2. **후방 패스(Backward pass):** 전방 패스의 집계 신호를 사용하여 모델의 가중치를 업데이트하는 과정입니다.

추론 중에는 전방 패스만 실행됩니다. 훈련 중에는 두 패스 모두 실행됩니다. 높은 수준에서, 후방 패스는 다음과 같이 작동합니다:

1. 전방 패스에서 계산된 출력과 예상된 출력(정답)을 비교합니다.  
   만약 두 값이 다르다면, 모델이 실수를 했다는 의미이며, 파라미터를 조정해야 합니다. 계산된 출력과 예상된 출력 간의 차이를 **손실(Loss)**이라고 합니다.

2. 각 학습 가능한 파라미터가 실수에 얼마나 기여했는지를 계산합니다.  
   이 값을 **기울기(Gradient)**라고 합니다. 수학적으로는 손실에 대해 각 학습 가능한 파라미터에 대한 미분 값을 계산합니다. 학습 가능한 파라미터마다 하나의 기울기 값이 있습니다. 기울기가 높은 파라미터는 손실에 크게 기여하므로 더 많이 조정해야 합니다.

3. 각 학습 가능한 파라미터 값을 해당 기울기에 따라 조정합니다.  
   각 파라미터를 얼마나 조정해야 하는지는 **최적화 도구(Optimizer)**에 의해 결정됩니다. 일반적으로 사용되는 최적화 도구로는 SGD(확률적 경사 하강법)와 Adam이 있으며, Transformer 기반 모델에서는 Adam이 가장 널리 사용됩니다.

**Figure 7-4**는 세 개의 파라미터와 하나의 비선형 활성화 함수가 있는 가상의 신경망에 대해 전방 및 후방 패스를 시각화한 것입니다. 이 더미 신경망은 시각화를 단순화하기 위해 사용되었습니다.

<img src="./images/fig_07_04.png" width="800">

후방 패스 동안, 각 학습 가능한 파라미터는 추가적인 값인 기울기와 최적화 상태를 함께 가집니다. 따라서 학습 가능한 파라미터가 많을수록 이러한 추가 값을 저장하는 데 더 많은 메모리가 필요합니다.

---


### **메모리 계산**

모델이 필요로 하는 메모리 양을 이해하는 것은 적절한 하드웨어를 선택하는 데 유용합니다. 종종 하드웨어를 이미 보유하고 있을 때 특정 모델을 실행할 수 있을지 계산해야 할 수도 있습니다. 예를 들어, 모델이 추론에 30GB의 메모리를 필요로 한다면, 24GB의 메모리를 가진 칩으로는 충분하지 않을 것입니다.

모델의 메모리 풋프린트는 모델 자체뿐만 아니라 작업 부하 및 메모리 사용량을 줄이기 위해 사용된 다양한 최적화 기술에 따라 달라집니다. 모든 최적화 기술과 작업 부하를 설명하는 것은 불가능하므로, 이 섹션에서는 근사 계산을 위한 공식만 소개합니다. 이를 통해 추론과 훈련 모두에서 모델을 운영하는 데 필요한 메모리를 대략적으로 이해할 수 있을 것입니다.

---

>**참고**
>
>추론과 훈련이 서로 다른 메모리 프로파일을 가지는 것은, 9장에서 논의된 대로, 추론 및 훈련용 칩이 서로 다른 이유 중 하나입니다.

---

**추론에 필요한 메모리**

추론 중에는 전방 패스(Forward Pass)만 실행됩니다. 전방 패스는 모델의 가중치를 위한 메모리를 필요로 합니다.  
- 모델의 파라미터 수를 **N**, 각 파라미터에 필요한 메모리를 **M**이라고 할 때, 모델의 파라미터를 로드하는 데 필요한 메모리는 다음과 같습니다:

$ N \times M $

전방 패스는 또한 활성화 값(Activation Values)을 위한 메모리를 필요로 합니다. Transformer 모델은 주의(attention) 메커니즘의 키-값(Key-Value) 벡터를 위한 메모리가 필요합니다. 활성화 값과 키-값 벡터의 메모리 사용량은 시퀀스 길이와 배치 크기에 따라 선형적으로 증가합니다.

많은 애플리케이션에서 활성화 값과 키-값 벡터의 메모리 사용량은 모델 가중치 메모리의 약 20%로 가정할 수 있습니다. 만약 애플리케이션이 더 긴 컨텍스트나 더 큰 배치 크기를 사용한다면, 실제로 필요한 메모리는 더 커질 것입니다. 이를 바탕으로 모델의 메모리 풋프린트는 다음과 같습니다:

$ N \times M \times 1.2 $

13B(130억) 파라미터 모델을 고려해 봅시다. 각 파라미터가 2바이트를 필요로 한다면, 모델의 가중치는 13B × 2바이트 = 26GB를 필요로 합니다. 추론에 필요한 총 메모리는 다음과 같습니다:

$ 26GB \times 1.2 = 31.2GB $

모델의 메모리 풋프린트는 모델 크기에 따라 빠르게 증가합니다. 모델이 커질수록 메모리는 이를 운영하는 데 병목 현상이 됩니다. 예를 들어, 파라미터 당 2바이트를 사용하는 70B(700억) 파라미터 모델은 가중치만으로도 무려 140GB의 메모리를 필요로 합니다.

---

**훈련에 필요한 메모리**  

모델을 훈련하려면 모델의 가중치와 활성화 값을 위한 메모리가 필요하며, 이 내용은 이미 논의되었습니다. 추가적으로, 학습 가능한 파라미터 수에 따라 확장되는 **기울기(gradients)**와 **최적화 상태(optimizer states)**를 위한 메모리도 필요합니다.

훈련에 필요한 총 메모리는 다음과 같이 계산됩니다:

$
\text{훈련 메모리} = \text{모델 가중치} + \text{활성화 값} + \text{기울기} + \text{최적화 상태}
$

---

>**팁**  
>
>후방 패스(Backward Pass) 동안, 각 학습 가능한 파라미터는 기울기 값을 위한 1개의 값과 최적화 상태에 따라 0~2개의 추가 값을 필요로 합니다:  
>- **기본 SGD(SGD optimizer)**는 최적화 상태를 저장하지 않습니다.  
>- **모멘텀 옵티마이저(Momentum optimizer)**는 학습 가능한 파라미터 당 1개의 값을 저장합니다.  
>- **Adam 옵티마이저**는 학습 가능한 파라미터 당 2개의 값을 저장합니다.

---

13B 파라미터 모델에서 모든 파라미터를 Adam 옵티마이저를 사용해 업데이트한다고 가정해봅시다.  
각 학습 가능한 파라미터가 기울기와 최적화 상태에 대해 3개의 값을 가지며, 각 값을 저장하는 데 2바이트가 필요하다면, 기울기와 최적화 상태에 필요한 메모리는 다음과 같습니다:

$
13\text{B} \times 3 \times 2\text{바이트} = 78\text{GB}
$

그러나 학습 가능한 파라미터가 1B(10억)개만 있다면, 필요한 메모리는 다음과 같이 줄어듭니다:

$
1\text{B} \times 3 \times 2\text{바이트} = 6\text{GB}
$


중요한 점은 이전 공식에서 활성화에 필요한 메모리가 모델 가중치에 필요한 메모리보다 적다고 가정했다는 것입니다. 그러나 실제로는 활성화 메모리가 훨씬 더 클 수 있습니다. 기울기 계산을 위해 활성화 값이 저장되는 경우, 활성화에 필요한 메모리는 모델 가중치에 필요한 메모리를 능가할 수 있습니다. Figure 7-5는 다양한 규모의 Megatron 모델에서 활성화에 필요한 메모리와 모델 가중치에 필요한 메모리를 비교한 내용을 보여줍니다. 이 자료는 Korthikanti et al.(2022)의 논문 *'Reducing Activation Recomputation in Large Transformer Models'*에서 인용되었습니다.

활성화 메모리를 줄이는 한 가지 방법은 활성화를 저장하지 않는 것입니다. 대신 필요할 때 활성화를 다시 계산(recompute)하는 방식입니다. 이 기술은 **기울기 체크포인팅(Gradient Checkpointing)** 또는 **활성화 재계산(Activation Recomputation)** 이라고 불립니다. 이는 메모리 요구량을 줄이지만, 재계산으로 인해 훈련 시간이 증가하는 단점이 있습니다.

<img src="./images/fig_07_05.png" width="800">

---


### **숫자 표현 방식**

지금까지의 메모리 계산에서 각 값이 2바이트의 메모리를 차지한다고 가정했습니다. 모델에서 각 값을 표현하는 데 필요한 메모리는 모델의 전체 메모리 풋프린트에 직접적으로 기여합니다. 각 값에 필요한 메모리를 절반으로 줄이면, 모델 가중치에 필요한 메모리도 절반으로 줄어듭니다.

각 값에 필요한 메모리를 줄이는 방법을 논의하기 전에, 숫자 표현 방식을 이해하는 것이 유용합니다. 신경망에서 숫자 값은 전통적으로 **부동 소수점(Float)**으로 표현됩니다. 가장 일반적인 부동 소수점 형식은 **FP(Floating Point)** 계열로, 이는 **IEEE(Institute of Electrical and Electronics Engineers)**의 부동 소수점 산술 표준(IEEE 754)을 따릅니다.

- **FP32**는 32비트(4바이트)를 사용해 부동 소수점을 표현하며, 이를 **단정밀도(Single Precision)**라고 합니다.
- **FP64**는 64비트(8바이트)를 사용하며, 이를 **배정밀도(Double Precision)**라고 합니다.
- **FP16**은 16비트(2바이트)를 사용하며, 이를 **반정밀도(Half Precision)**라고 합니다.

FP64는 여전히 많은 계산에서 사용되지만(이 글 작성 시점 기준으로 NumPy와 pandas의 기본 형식), 신경망에서는 메모리 풋프린트 문제로 인해 거의 사용되지 않습니다. FP32와 FP16이 더 일반적입니다. AI 작업에서 널리 사용되는 다른 부동 소수점 형식으로는 **BF16(BFloat16)**과 **TF32(TensorFloat-32)**가 있습니다. BF16은 Google이 TPU 성능 최적화를 위해 설계했으며, TF32는 NVIDIA가 GPU 성능을 최적화하기 위해 설계했습니다.

숫자는 정수로도 표현될 수 있습니다. 부동 소수점 형식만큼 일반적이지는 않지만, 정수 표현 방식은 점점 더 인기를 얻고 있습니다. 일반적인 정수 형식으로는 **INT8(8비트 정수)**와 **INT4(4비트 정수)**가 있습니다.

각 부동 소수점 형식은 일반적으로 숫자의 부호(양수 또는 음수)를 나타내는 데 1비트를 사용합니다. 나머지 비트는 **범위(Range)**와 **정밀도(Precision)**로 나뉩니다.

- **범위(Range)**  
  범위 비트의 수는 해당 형식이 표현할 수 있는 값의 범위를 결정합니다. 더 많은 비트는 더 넓은 범위를 의미합니다. 이는 더 많은 자릿수를 사용할수록 더 넓은 범위를 표현할 수 있는 것과 유사합니다.

- **정밀도(Precision)**  
  정밀도 비트의 수는 숫자를 얼마나 정확하게 표현할 수 있는지를 결정합니다. 정밀도 비트 수를 줄이면 숫자의 정확도가 떨어집니다. 예를 들어, 10.1234를 소수점 두 자리까지만 지원하는 형식으로 변환하면 이 값은 10.12로 변환되어 원래 값보다 덜 정확하게 표현됩니다.

**Figure 7-6**은 부동 소수점 형식의 범위 및 정밀도 비트와 함께 다양한 부동 소수점 형식을 보여줍니다.

<img src="./images/fig_07_06.png" width="800">

**비트 수가 많은 형식은 더 높은 정밀도(higher precision)로 간주됩니다.**  
높은 정밀도 형식을 낮은 정밀도 형식(예: FP32에서 FP16으로)으로 변환하는 것은 정밀도를 줄이는 것을 의미합니다. 정밀도를 줄이면 값이 변경되거나 오류가 발생할 수 있습니다. **Table 7-3**은 FP32 값을 FP16, BF16, TF32로 변환한 예와 결과적인 부정확성을 보여줍니다.

**Table 7-3. FP32 값을 낮은 정밀도 형식으로 변환한 결과.**  
**부정확성은 기울임꼴로 표시됨.**

| FP32 값          | FP16 값                  | BF16 값   | TF32 값                |
|-------------------|--------------------------|-----------|------------------------|
| 0.0123456789      | 0.0123443603515625       | 0.0123291 | 0.0123443603515625     |
| 0.123456789       | 0.12347412109375         | 0.123535  | 0.1234130859375        |
| 1.23456789        | 1.234375                 | 1.23438   | 1.234375               |
| 12.3456789        | 12.34375                 | 12.375    | 12.34375               |
| 123.456789        | 123.4375                 | 123.5     | 123.4375               |
| 1234.56789        | 1235.0                   | 1232.0    | 1234.0                 |
| 12345.6789        | 12344.0                  | 12352.0   | 12344.0                |
| 123456.789        | **INF**                  | 123392.0  | 123456.0               |
| 1234567.89        | **INF**                  | 1236990.0 | 1233920.0              |

Table 7-3에서 알 수 있듯이, **BF16**과 **FP16**은 동일한 비트 수를 가지고 있지만, BF16은 **범위(range)**에 더 많은 비트를 할당하고 **정밀도(precision)**에는 적은 비트를 할당합니다.  
이로 인해 BF16은 FP16에서 범위를 초과하는 큰 값을 표현할 수 있습니다. 그러나 이 때문에 BF16은 FP16보다 정밀도가 낮아집니다.  
예를 들어, `1234.56789`는 FP16에서는 `1235.0`(0.035% 값 변화)로 변환되지만, BF16에서는 `1232.0`(0.208% 값 변화)로 변환됩니다.

---

>**경고**  
>
>모델을 사용할 때, 모델이 설계된 형식으로 로드되었는지 확인해야 합니다. 잘못된 숫자 형식으로 모델을 로드하면 모델 성능이 크게 변화할 수 있습니다. 예를 들어, Llama 2는 출시 시 가중치가 BF16으로 설정되어 있었습니다. 그러나 많은 팀이 이 모델을 FP16 형식으로 로드했으며, 이로 인해 모델의 품질이 광고된 것보다 훨씬 낮아지는 문제를 경험했습니다. 이러한 오해로 인해 많은 사람들이 시간을 낭비했지만, 긍정적인 측면은 숫자 표현 방식에 대한 이해를 높이는 계기가 되었다는 점입니다.

적절한 형식의 선택은 작업의 수치 분포(필요한 값의 범위 등), 작은 수치 변화에 대한 작업의 민감도, 그리고 사용하는 하드웨어에 따라 달라집니다.

---

### **양자화(Quantization)**

모델의 값을 표현하는 데 필요한 비트 수가 적을수록 모델의 메모리 풋프린트가 줄어듭니다. 32비트 형식의 10B 파라미터 모델은 가중치를 위해 40GB가 필요하지만, 동일한 모델을 16비트 형식으로 표현하면 20GB만 필요합니다. 정밀도 감소(reducing precision), 즉 양자화(quantization)는 모델의 메모리 풋프린트를 줄이는 저렴하고 매우 효과적인 방법입니다. 이 방법은 간단하며 다양한 작업과 아키텍처에 일반적으로 적용됩니다. **머신러닝**에서 저정밀도(low precision)는 일반적으로 FP32보다 적은 비트를 사용하는 모든 형식을 지칭합니다.

---

>**양자화와 정밀도 감소의 차이(QUANTIZATION VERSUS REDUCED PRECISION)**
>
>엄밀히 말하면, 양자화는 대상 형식이 정수(integer)일 때만 해당됩니다. 그러나 실제로 양자화라는 용어는 값을 저정밀도 형식으로 변환하는 모든 기술을 지칭하는 데 사용됩니다. 이 책에서는 문헌과의 일관성을 유지하기 위해 **정밀도 감소(precision reduction)**를 양자화로 지칭합니다.

---

양자화를 수행하려면, **무엇을 양자화할지**와 **언제 양자화할지**를 결정해야 합니다.

- **무엇을 양자화할 것인가(What to quantize)**
    - 이상적으로는 메모리를 가장 많이 소비하는 것을 양자화해야 하지만, 성능에 큰 영향을 주지 않으면서 양자화할 수 있는 것을 선택해야 합니다. *"Memory Math"*에서 논의된 바와 같이, 추론 중 모델의 메모리 풋프린트에 가장 크게 기여하는 요소는 **모델의 가중치(weights)**와 **활성화 값(activations)**입니다. 가중치 양자화(weight quantization)는 활성화 양자화(activation quantization)보다 더 일반적입니다. 이는 가중치 양자화가 성능에 미치는 영향을 더 안정적으로 유지하면서 정확도 손실이 적기 때문입니다.

- **언제 양자화할 것인가(When to quantize)**
    - 양자화는 훈련 중 또는 훈련 후(post-training)에 수행될 수 있습니다. **훈련 후 양자화(PTQ)**는 모델이 완전히 훈련된 후에 양자화를 수행하는 것을 의미합니다. PTQ는 가장 일반적인 방법입니다. 특히 모델을 직접 훈련하지 않는 **AI 애플리케이션 개발자**들에게 더 관련성이 있습니다.

---

**추론에서의 양자화(Inference quantization)**

딥러닝 초기에는 32비트 FP32를 사용해 모델을 훈련하고 서비스하는 것이 표준이었습니다. 그러나 2010년대 후반 이후, 16비트, 심지어는 더 낮은 정밀도로 모델을 서비스하는 것이 점점 더 일반화되었습니다. 예를 들어, **Dettmers et al.(2022)**는 LLMs를 8비트로 양자화하는 데 탁월한 작업을 수행했으며, **QLoRA(Dettmers et al., 2023)**를 사용해 4비트로도 구현했습니다.

모델은 혼합 정밀도(mixed precision)에서도 서비스될 수 있습니다. 가능한 경우 정밀도를 줄이고, 필요한 경우 높은 정밀도를 유지하는 방식입니다. Apple(2024)은 2비트와 4비트 형식을 혼합하여 평균 3.5비트-가중치를 사용하는 양자화 방식을 도입했습니다. 또한 NVIDIA는 2024년에 4비트 신경망을 예상하며, 4비트 부동 소수점 추론을 지원하는 **Blackwell** GPU 아키텍처를 발표했습니다.

8비트 이하로 이동하면 숫자 표현이 더욱 복잡해집니다. FP8(8비트)과 FP4(4비트)와 같은 **미니플로트(minifloat)** 형식을 사용해 가중치를 부동 소수점으로 유지할 수 있습니다. 그러나 더 일반적으로는 INT8 또는 INT4와 같은 정수 형식으로 변환됩니다.

양자화는 효과적이지만 한계가 있습니다. 값당 1비트 미만으로 줄일 수는 없으며, 일부 연구에서는 1비트 표현을 시도했습니다. 예를 들어, **BinaryConnect(Courbariaux et al., 2015)**, **Xnor-Net(Rastegari et al., 2016)**, **BitNet(Wang et al., 2023)**이 있습니다.

2024년, Microsoft 연구진(Ma et al.)은 **BitNet b1.58**을 도입하며 **1비트 LLM의 시대**가 도래했음을 선언했습니다. BitNet b1.58은 1.58비트/파라미터만 필요로 하며, 최대 3.9B 파라미터에서 16비트 **Llama 2(Touvron et al., 2023)**와 유사한 성능을 보입니다. 이는 **Table 7-4**에 자세히 나와 있습니다.

**Table 7-4. BitNet b1.58의 성능 비교: 다양한 벤치마크 및 모델 크기(최대 3.9B 파라미터)에서 Llama 2 16비트와의 성능 비교. Ma et al.(2024)의 결과.**

| **모델(Model)** | **크기(Size)** | **ARCe** | **ARCc** | **HS** | **BQ** | **OQ** | **PQ** | **WGe** | **평균(Avg.)** |
|------------------|----------------|----------|----------|--------|--------|--------|--------|--------|--------------|
| Llama LLM       | 700M           | 54.7     | 23.0     | 37.0   | 60.0   | 20.2   | 68.9   | 54.8   | 45.5         |
| BitNet b1.58    | 700M           | 51.8     | 21.4     | 35.1   | 59.2   | 20.0   | 68.1   | 55.2   | 44.3         |
| Llama LLM       | 1.3B           | 56.9     | 23.5     | 38.5   | 59.1   | 21.6   | 70.0   | 53.9   | 46.2         |
| BitNet b1.58    | 1.3B           | 54.9     | 24.2     | 37.7   | 59.7   | 19.6   | 68.8   | 55.8   | 45.4         |
| Llama LLM       | 3B             | 62.1     | 25.6     | 43.3   | 61.8   | 24.6   | 72.1   | 58.2   | 49.7         |
| BitNet b1.58    | 3B             | 61.4     | 28.3     | 42.9   | 61.5   | 26.6   | 71.5   | 59.3   | 50.2         |
| BitNet b1.58    | 3.9B           | 64.2     | 28.7     | 44.2   | 63.5   | 24.2   | 73.2   | 60.5   | 51.2         |

정밀도를 낮추는 것은 메모리 풋프린트를 줄일 뿐만 아니라, 종종 계산 속도를 개선하기도 합니다. 첫째, 더 큰 배치 크기를 허용하여 모델이 더 많은 입력을 병렬로 처리할 수 있게 합니다. 둘째, 정밀도를 낮추면 계산 속도가 빨라져 추론 지연 시간과 훈련 시간을 더욱 단축할 수 있습니다. 이를 설명하기 위해 두 숫자를 더하는 예를 들어 보겠습니다. 덧셈을 비트 단위로 수행한다고 가정할 때, 각 비트가 t 나노초가 걸린다면, 32비트의 경우 32t 나노초가 걸리지만, 16비트의 경우 16t 나노초만 걸립니다. 그러나 정밀도를 낮추는 것이 항상 지연 시간을 줄이는 것은 아닙니다. 이는 형식 변환에 필요한 추가 계산 때문입니다.

정밀도를 낮추는 것에는 단점이 있습니다. 각 변환은 종종 작은 값의 변화를 유발하며, 많은 작은 변화가 큰 성능 변화를 초래할 수 있습니다. 만약 값이 낮아진 정밀도 형식이 표현할 수 있는 범위를 벗어나면, 해당 값은 무한대나 임의의 값으로 변환될 수 있으며, 이는 모델 품질의 추가적인 저하를 초래할 수 있습니다. 모델 성능에 최소한의 영향을 주면서 정밀도를 줄이는 방법은 모델 개발자, 하드웨어 제조업체, 애플리케이션 개발자들이 활발히 연구하는 분야입니다.

낮은 정밀도로의 추론은 이제 표준이 되었습니다. 모델은 성능을 극대화하기 위해 높은 정밀도 형식을 사용하여 훈련된 후, 추론 시에는 정밀도가 낮아집니다. PyTorch, TensorFlow, Hugging Face의 transformers와 같은 주요 ML 프레임워크는 몇 줄의 코드로 PTQ를 무료로 제공합니다.

일부 엣지 디바이스는 양자화된 추론만 지원합니다. 따라서 TensorFlow Lite와 PyTorch Mobile과 같은 온디바이스 추론 프레임워크도 PTQ를 제공합니다.

---

**학습 양자화**

학습 중 양자화는 아직 PTQ만큼 보편화되지는 않았지만 점차 주목받고 있습니다. 학습 양자화에는 두 가지 주요 목표가 있습니다.

1. 낮은 정밀도에서 양질의 추론 성능을 제공하는 모델을 생성하는 것. 이는 학습 후 양자화(Post-Training Quantization) 과정에서 모델 품질이 저하되는 문제를 해결하는 데 목적이 있습니다.

2. 학습 시간과 비용을 줄이는 것. 양자화는 모델의 메모리 사용량을 줄여 더 저렴한 하드웨어에서 모델을 학습하거나 동일한 하드웨어에서 더 큰 모델을 학습할 수 있도록 합니다. 또한 계산 속도를 높여 비용을 추가로 절감합니다.

양자화 기술은 이러한 목표 중 하나 또는 둘 다 달성하는 데 도움을 줄 수 있습니다.

**양자화 인식 학습(QAT)**은 낮은 정밀도 추론에서 높은 품질을 유지할 수 있는 모델을 생성하는 것을 목표로 합니다. QAT는 학습 중 낮은 정밀도(예: 8비트) 동작을 시뮬레이션하여 모델이 낮은 정밀도에서 높은 품질의 출력을 생성하도록 학습합니다. 그러나 QAT는 학습 시간 자체를 줄이지 않으며, 낮은 정밀도 동작을 시뮬레이션하는 추가 작업으로 인해 학습 시간이 증가할 수도 있습니다.

반면, 모델을 직접 낮은 정밀도로 학습하는 것은 두 가지 목표를 모두 달성하는 데 도움이 될 수 있습니다. 2016년 Hubara 외(2016) 및 Jacob 외(2017) 이후로 모델을 낮은 정밀도로 학습하려는 시도가 이루어졌습니다. Character.AI(2024)는 INT8로만 모델을 학습하여 학습/서빙 간 정밀도 불일치를 제거하고 학습 효율성을 크게 향상시킬 수 있었다고 발표했습니다. 그러나 낮은 정밀도에서 학습은 더 어려운 작업으로, 역전파가 낮은 정밀도에 더 민감하기 때문입니다.

낮은 정밀도 학습은 종종 혼합 정밀도(Mixed Precision)로 수행되며, 이 경우 가중치의 복사본은 높은 정밀도로 유지되지만 그래디언트와 활성화 값과 같은 다른 값은 낮은 정밀도로 유지됩니다. 예를 들어, LLM-QAT(Liu 외, 2023)는 가중치와 활성화 값을 4비트로 양자화하지만, 임베딩은 16비트로 유지합니다.

모델에서 낮은 정밀도로 유지해야 하는 부분은 많은 ML 프레임워크에서 제공하는 **자동 혼합 정밀도(Automatic Mixed Precision, AMP)** 기능을 사용하여 자동으로 설정할 수 있습니다.

또한, 학습의 다양한 단계를 서로 다른 정밀도 수준에서 수행하는 것도 가능합니다. 예를 들어, 모델을 높은 정밀도로 학습한 후 낮은 정밀도로 파인튜닝할 수 있습니다. 이는 특히 **파운데이션 모델**에서 일반적인 방식으로, 처음부터 모델을 학습하는 팀은 높은 정밀도 학습을 위한 충분한 컴퓨팅 자원을 보유한 조직일 수 있습니다. 반면, 모델이 공개된 후에는 더 적은 컴퓨팅 자원을 가진 개발자들이 낮은 정밀도로 해당 모델을 파인튜닝할 수 있습니다.

---

## **파인튜닝 기술**  
이전 섹션을 통해 대규모 모델을 파인튜닝하는 작업이 얼마나 많은 메모리를 요구하는지 명확히 이해했기를 바랍니다. 파인튜닝에 더 많은 메모리가 필요할수록 이를 수행할 수 있는 사람은 줄어듭니다. 따라서 모델의 메모리 사용량을 줄이는 기술은 파인튜닝의 접근성을 높이고, 더 많은 사람들이 모델을 자신만의 애플리케이션에 맞게 조정할 수 있도록 만듭니다. 이 섹션은 **메모리 효율적인 파인튜닝** 기술에 중점을 두며, 그 중에서도 **매개변수 효율적인 파인튜닝(Parameter-Efficient Finetuning)** 에 초점을 맞춥니다.

또한, **모델 병합(Model Merging)** 이라는 흥미롭지만 더 실험적인 접근 방식을 다룰 것입니다. 모델 병합은 일반적으로 파인튜닝으로 간주되지는 않지만, 파인튜닝과 상호보완적인 관계이기 때문에 이 섹션에 포함했습니다. 파인튜닝은 특정 요구에 맞춰 하나의 모델을 조정하는 작업이고, 모델 병합은 여러 모델(종종 파인튜닝된 모델)을 동일한 목적을 위해 결합하는 방식입니다.

여러 모델을 결합하는 것이 새로운 개념은 아니지만, 새로운 유형의 모델과 파인튜닝 기술이 등장하면서 창의적인 모델 병합 기법들이 많이 탄생하게 되었습니다. 이 섹션을 작성하는 것이 특히 재미있었던 이유이기도 합니다.

---

### **매개변수 효율적인 파인튜닝(Parameter-Efficient Finetuning)**  
파인튜닝 초창기에는 모델이 충분히 작아서 전체 모델을 파인튜닝할 수 있었습니다. 이러한 접근 방식은 **전체 파인튜닝(Full Finetuning)**이라고 불립니다. 전체 파인튜닝에서는 학습 가능한 매개변수의 수가 모델의 전체 매개변수 수와 동일합니다.

전체 파인튜닝은 학습과 비슷해 보일 수 있습니다. 주요 차이점은 학습은 무작위 초기화된 모델 가중치로 시작하는 반면, 파인튜닝은 이미 학습된 모델 가중치로 시작한다는 점입니다.

“메모리 계산(Memory Math)”에서 논의했듯이, 학습 가능한 매개변수 수가 많을수록 더 많은 메모리가 필요합니다. 예를 들어, 70억(7B) 개의 매개변수를 가진 모델을 고려해보겠습니다.

- FP16과 같은 16비트 형식을 사용할 경우, 모델의 가중치를 로드하는 데만 14GB의 메모리가 필요합니다.  
- Adam 옵티마이저를 사용하여 이 모델을 전체 파인튜닝하려면, 동일한 16비트 형식으로 70억 × 3 × 2 바이트 = 42GB의 추가 메모리가 필요합니다.  
- 따라서 모델의 가중치, 그래디언트, 옵티마이저 상태를 위한 총 메모리 요구량은 14GB + 42GB = **56GB**입니다.

56GB는 대부분의 소비자용 GPU 메모리 용량을 초과하며, 소비자용 GPU는 일반적으로 12~24GB의 메모리를 제공하고, 고급형 GPU는 최대 48GB를 제공합니다. 그리고 이 메모리 추정치는 활성화(activations)에 필요한 메모리는 포함하지 않은 수치입니다.

---

>**주의**
>
>주어진 하드웨어에 모델을 맞추기 위해서는 모델의 메모리 사용량을 줄이거나 하드웨어 메모리를 더 효율적으로 사용하는 방법을 찾아야 합니다. **양자화(Quantization)**와 **매개변수 효율적 파인튜닝(PEFT)** 같은 기술은 전체 메모리 사용량을 최소화하는 데 도움을 줍니다. 하드웨어 메모리를 더 효율적으로 사용하는 데 초점을 맞춘 기술에는 **CPU 오프로딩(CPU Offloading)**이 포함됩니다. 전체 모델을 GPU에 적재하려고 시도하는 대신, 초과 메모리를 CPU로 오프로딩할 수 있습니다. 이는 DeepSpeed(Rasley 외, 2020)가 보여준 바와 같이 효과적인 방법입니다.

___

**우리는 아직 전체 파인튜닝(Full Finetuning), 특히 감독(supervised) 파인튜닝과 선호(preference) 파인튜닝이 고품질의 주석이 달린 데이터가 많이 필요하다는 점에 대해서는 다루지 않았습니다.** 이러한 데이터는 대부분의 사람들이 감당할 수 없을 만큼 비용이 많이 듭니다. 전체 파인튜닝의 높은 메모리와 데이터 요구사항 때문에 사람들은 **부분 파인튜닝(Partial Finetuning)**을 수행하기 시작했습니다. 부분 파인튜닝에서는 모델의 일부 매개변수만 업데이트됩니다. 예를 들어, 모델에 10개의 레이어가 있을 경우, 처음 9개의 레이어를 고정(freeze)하고 마지막 레이어만 파인튜닝하면, 학습 가능한 매개변수의 수를 전체 파인튜닝의 10%로 줄일 수 있습니다.  

부분 파인튜닝은 메모리 사용량을 줄일 수 있지만, **매개변수 효율성(parameter efficiency)이 떨어집니다.** 부분 파인튜닝은 전체 파인튜닝과 비슷한 성능을 달성하기 위해 여전히 많은 학습 가능한 매개변수를 필요로 합니다. Houlsby 외(2019)의 연구에 따르면, BERT large(Devlin 외, 2018) 모델에서 GLUE 벤치마크(Wang 외, 2018)의 성능을 전체 파인튜닝과 유사하게 만들려면 매개변수의 약 25%를 업데이트해야 한다고 합니다. **그림 7-7**은 학습 가능한 매개변수의 수에 따라 부분 파인튜닝의 성능 곡선을 보여줍니다.  

<img src="./images/fig_07_07.png" width="800">

**이 질문이 제기됩니다: 어떻게 학습 가능한 매개변수를 대폭 줄이면서도 전체 파인튜닝(Full Finetuning)에 가까운 성능을 달성할 수 있을까?** 이 질문에서 파생된 파인튜닝 기술은 **매개변수 효율적(Parameter-Efficient)**으로 간주됩니다. 매개변수 효율적인지 여부를 판단할 명확한 기준은 없지만, 일반적으로 학습 가능한 매개변수를 몇 배 적게 사용하면서도 전체 파인튜닝에 가까운 성능을 달성할 수 있다면 매개변수 효율적인 기술로 간주됩니다.

**매개변수 효율적 파인튜닝(PEFT)**의 개념은 Houlsby 외(2019)에 의해 처음 제안되었습니다. 연구자들은 모델에 추가 매개변수를 적절한 위치에 삽입함으로써 적은 수의 학습 가능한 매개변수로도 강력한 파인튜닝 성능을 달성할 수 있음을 보여주었습니다. 이들은 BERT 모델의 각 Transformer 블록에 두 개의 **어댑터 모듈(Adapter Modules)**을 삽입하였으며, 이는 **그림 7-8**에 나타나 있습니다.

<img src="./images/fig_07_08.png" width="800">

**파인튜닝 과정에서, 연구자들은 모델의 원래 매개변수를 변경하지 않고 어댑터만 업데이트했습니다.** 학습 가능한 매개변수의 수는 어댑터 내 매개변수의 수와 동일합니다. GLUE 벤치마크에서, 이들은 학습 가능한 매개변수 수를 전체 파인튜닝의 3%로 줄이면서도 전체 파인튜닝 대비 0.4% 이내의 성능을 달성했습니다. **그림 7-7**의 주황색 선은 전체 파인튜닝과 서로 다른 어댑터 크기를 사용하는 파인튜닝 간의 성능 차이를 보여줍니다.

그러나 이 접근 방식의 단점은 **파인튜닝된 모델의 추론 지연 시간(inference latency)이 증가**한다는 점입니다. 어댑터가 추가 레이어를 도입함으로써, 순방향 패스(forward pass)에 더 많은 계산 단계가 추가되고, 이는 추론 속도를 저하시킵니다.

**PEFT**는 더 저렴한 하드웨어에서도 파인튜닝이 가능하게 하여 더 많은 개발자가 접근할 수 있도록 만듭니다. PEFT 기법은 일반적으로 매개변수 효율적일 뿐만 아니라 **데이터 효율적(sample-efficient)**이기도 합니다. 전체 파인튜닝은 수만에서 수백만 개의 예제가 필요할 수 있는 반면, 일부 PEFT 기법은 단 몇 천 개의 예제만으로도 강력한 성능을 발휘할 수 있습니다.

PEFT의 명확한 매력 덕분에 PEFT 기술은 빠르게 개발되고 있습니다. 다음 섹션에서는 이러한 기술의 개요를 제공한 후, 가장 일반적인 PEFT 기술인 **LoRA**에 대해 더 깊이 다룰 것입니다.

---

이미지의 내용을 텍스트로 추출한 후 요청하신 대로 한국어로 번역하겠습니다. 잠시만 기다려 주세요.

**PEFT 기법**

현재 **매개변수 효율적 파인튜닝(PEFT)** 분야는 크게 두 가지 범주로 나뉩니다: **어댑터 기반 기법(Adapter-based methods)**과 **소프트 프롬프트 기반 기법(Soft Prompt-based methods)**입니다. 하지만 앞으로 새로운 범주가 추가될 가능성도 있습니다.

**어댑터 기반 기법**은 Houlsby 외(2019)에 의해 개발된 방법처럼, 모델의 가중치에 추가적인 모듈을 삽입하는 모든 방법을 포함합니다. 어댑터 기반 기법은 매개변수를 추가하기 때문에 **가산적 방법(Additive Methods)**이라고도 불립니다.

현재, **LoRA(Hu 외, 2021)**는 가장 인기 있는 어댑터 기반 기법이며, 다음 섹션에서 다룰 예정입니다. 다른 어댑터 기반 기법으로는 LoRA와 비슷한 시기에 발표된 **BitFit(Zaken 외, 2021)**이 있습니다. 더 최근의 어댑터 방법으로는 **IA3(Liu 외, 2022)**이 있는데, 이 방법은 효율적인 혼합 작업 배칭 전략을 사용하여 다중 작업 파인튜닝에 특히 매력적입니다. 일부 사례에서는 LoRA와 전체 파인튜닝보다 더 우수한 성능을 보여줍니다. 또한, **LongLoRA(Chen 외, 2023)**는 컨텍스트 길이를 확장하기 위해 주의 수정(attention-modification) 기술을 통합한 LoRA의 변형입니다.

**어댑터 기반 기법**이 모델 구조에 학습 가능한 매개변수를 추가한다면, **소프트 프롬프트 기반 기법**은 입력 처리를 변경하여 학습 가능한 특수 토큰을 도입합니다. 이 추가된 토큰은 입력 토큰과 함께 모델에 입력됩니다. **소프트 프롬프트**는 입력(hard prompt)처럼 모델의 행동을 안내하지만, 다음 두 가지 점에서 하드 프롬프트와 다릅니다:

1. **하드 프롬프트**는 사람이 읽을 수 있습니다. 일반적으로 "I", "write", "a", "lot"와 같은 이산적인(discrete) 토큰을 포함합니다. 반면, **소프트 프롬프트**는 연속적인 벡터로, 임베딩 벡터와 유사하며 사람이 읽을 수 없습니다.
2. 하드 프롬프트는 정적이고 학습할 수 없지만, 소프트 프롬프트는 튜닝 과정에서 역전파(backpropagation)를 통해 최적화할 수 있어 특정 작업에 맞게 조정이 가능합니다.

일부 사람들은 소프트 프롬프트를 **프롬프트 엔지니어링과 파인튜닝의 교차점**으로 묘사합니다. **그림 7-9**는 소프트 프롬프트와 하드 프롬프트를 함께 사용하여 모델의 행동을 안내하는 방법을 시각화합니다.

<img src="./images/fig_07_09.png" width="800">

**소프트 프롬프트 튜닝(Soft Prompt Tuning)**은 혼란스러울 수 있는 여러 유사한 기법들로 특징지어집니다. 예를 들어, **프리픽스 튜닝(Prefix-Tuning)**(Li and Liang, 2021), **P-Tuning**(Liu 외, 2021), 그리고 **프롬프트 튜닝(Prompt Tuning)**(Lester 외, 2021)이 있습니다. 이들은 주로 소프트 프롬프트가 삽입되는 위치에서 차이가 납니다. 예를 들어, 프리픽스 튜닝은 소프트 프롬프트 토큰을 모든 Transformer 레이어의 입력 앞에 추가합니다. 반면, 프롬프트 튜닝은 소프트 프롬프트 토큰을 임베딩된 입력 앞에만 추가합니다. 이러한 기법을 사용하고자 한다면, 많은 PEFT 프레임워크가 이를 기본적으로 구현하여 제공합니다.

PEFT 기법이 실제로 어떻게 사용되고 있는지 이해하기 위해, 2024년 10월에 GitHub 저장소 **huggingface/peft**에서 1,000개 이상의 오픈 이슈를 분석했습니다. 가정은 어떤 기법을 사용하는 경우, 해당 기법에 대해 문제를 보고하거나 질문할 가능성이 높다는 것입니다. **그림 7-10**은 그 결과를 보여줍니다. "P-Tuning"에 대해서는 다양한 철자 변형을 고려하기 위해 "p_tuning"과 "p tuning" 키워드로 검색했습니다.

<img src="./images/fig_07_10.png" width="800">

이 분석에 따르면, **LoRA**가 압도적으로 사용되고 있음을 알 수 있습니다. 소프트 프롬프트는 상대적으로 덜 일반적이지만, 프롬프트 엔지니어링보다 더 많은 커스터마이징을 원하면서도 파인튜닝에 많은 시간을 투자하고 싶지 않은 사람들 사이에서 관심이 점차 높아지고 있는 것으로 보입니다.

LoRA의 인기로 인해, 다음 섹션에서는 LoRA가 어떻게 작동하며 초기 어댑터 기반 기법의 문제를 어떻게 해결했는지에 대해 집중적으로 다룰 예정입니다. LoRA를 사용하지 않더라도, 이 심층 분석은 다른 파인튜닝 방법을 탐구하는 데 유용한 프레임워크를 제공할 것입니다.

---

**LoRA**

Houlsby 외(2019)가 제안한 원래의 어댑터 방법과는 달리, **LoRA(Low-Rank Adaptation)**(Hu 외, 2021)은 추가적인 추론 지연 시간을 초래하지 않는 방식으로 매개변수를 추가합니다. 기본 모델에 추가적인 레이어를 도입하는 대신, LoRA는 원래의 레이어로 병합할 수 있는 모듈을 사용합니다.

LoRA는 개별 가중치 행렬에 적용할 수 있습니다. 주어진 가중치 행렬을 LoRA는 두 개의 더 작은 행렬로 분해한 다음, 이를 업데이트하고 원래의 행렬로 병합합니다.

다음은 차원이 $ n \times m $인 가중치 행렬 $ W $에 LoRA를 적용하는 과정입니다:

1. **작은 행렬의 차원을 선택합니다.** 선택한 값을 $ r $라고 하겠습니다. 두 개의 행렬 $ A $(차원 $ n \times r $)와 $ B $(차원 $ r \times m $)을 생성합니다. 이들의 곱은 $ W_{AB} $이며, 이는 $ W $와 동일한 차원을 가집니다. 여기서 $ r $은 LoRA의 랭크(rank)입니다.

2. **$ W_{AB} $를 원래의 가중치 행렬 $ W $에 추가하여 새로운 가중치 행렬 $ W' $를 생성합니다.** 모델의 일부로 $ W $ 대신 $ W' $를 사용합니다. 하이퍼파라미터 $ \alpha $를 사용하여 $ W_{AB} $가 새로운 행렬에 얼마나 기여해야 하는지 결정할 수 있습니다. 수식은 다음과 같습니다:  
   $
   W' = W + \frac{\alpha}{r} W_{AB}
   $

3. **파인튜닝 중에는 $ A $와 $ B $의 매개변수만 업데이트합니다.** $ W $는 그대로 유지됩니다.

**그림 7-11**은 이 과정을 시각화합니다.

<img src="./images/fig_07_11.png" width="800">

---

>**주의**
>
>**LoRA(Low-Rank Adaptation)**은 오랜 역사를 가진 차원 축소 기법인 **저랭크 행렬 분해(Low-Rank Factorization)** 개념에 기반을 두고 있습니다. 핵심 아이디어는 큰 행렬을 두 개의 더 작은 행렬의 곱으로 분해하여 매개변수의 수를 줄이는 것입니다. 이는 계산 및 메모리 요구사항을 모두 감소시키는 효과를 가져옵니다. 예를 들어, $ 9 \times 9 $ 행렬은 $ 9 \times 1 $과 $ 1 \times 9 $ 크기의 두 행렬로 분해될 수 있습니다. 원래 행렬은 81개의 매개변수를 가지지만, 분해된 두 행렬은 총 18개의 매개변수만 가집니다.
>
>첫 번째 분해된 행렬의 열 수와 두 번째 분해된 행렬의 열 수는 분해의 **랭크(rank)**에 해당합니다. 원래 행렬은 **풀랭크(full-rank)**이며, 두 작은 행렬은 원래 행렬의 저랭크 근사(low-rank approximation)를 나타냅니다.
>
> 행렬 분해는 매개변수의 수를 크게 줄일 수 있지만, **손실(lossy)**이 발생합니다. 이는 원래 행렬을 근사화하기 때문입니다. **랭크가 높을수록** 분해가 원래 행렬의 정보를 더 많이 보존할 수 있습니다.

---

원래의 어댑터 방식처럼, **LoRA**는 **매개변수 효율적(parameter-efficient)**이고 **데이터 효율적(sample-efficient)**입니다. 행렬 분해를 통해 LoRA는 더 적은 학습 가능한 매개변수를 사용할 수 있습니다. **LoRA 논문**에 따르면, GPT-3 모델의 경우, LoRA는 전체 파인튜닝에 비해 약 **470만 개(4.7M)**의 학습 가능한 매개변수만 사용하면서도 여러 작업에서 **동등하거나 더 나은 성능**을 달성했습니다. 이는 전체 파인튜닝의 **0.0027%**에 해당합니다.

---

**왜 LoRA가 효과적인가?**

**LoRA**와 같은 매개변수 효율적인 방법은 매우 인기를 끌고 있지만, **매개변수 효율성이 왜 가능한지**에 대한 의문이 남습니다. 만약 모델이 사전 학습 중 특정 행동을 학습하는 데 많은 매개변수를 필요로 한다면, 파인튜닝 시 행동을 변경하기 위해서도 많은 매개변수가 필요하지 않을까요?

데이터에 대해서도 동일한 질문을 할 수 있습니다. 모델이 특정 행동을 학습하기 위해 많은 데이터를 필요로 한다면, 그 행동을 의미 있게 변경하기 위해서도 많은 데이터가 필요하지 않을까요? 어떻게 수백만 또는 수십억 개의 예제가 사전 학습에는 필요하지만, 파인튜닝에는 몇백 개 또는 몇천 개의 예제만으로 충분할 수 있을까요?

많은 연구는 LLM(Large Language Model)이 많은 매개변수를 가지고 있지만, **내재적 차원(intrinsic dimension)**이 매우 낮다고 주장합니다(Li 외, 2018; Aghajanyan 외, 2020; Hu 외, 2021). 이들은 사전 학습이 모델의 내재적 차원을 암묵적으로 최소화한다고 보여줍니다. 놀랍게도, 사전 학습 후 더 큰 모델은 더 낮은 내재적 차원을 가지는 경향이 있습니다. 이는 사전 학습이 다운스트림 작업을 위한 **압축 프레임워크**로 작용함을 시사합니다. 다시 말해, 더 잘 학습된 LLM일수록 적은 수의 학습 가능한 매개변수와 적은 데이터로 파인튜닝하기가 더 쉬워집니다.

**LoRA를 사전 학습에도 사용할 수 있을까?** 저랭크 행렬 분해(low-rank factorization)가 매우 효과적이라면, 왜 이를 사전 학습에도 사용하지 않을까요? 대규모 모델을 사전 학습하고 파인튜닝 시에만 저랭크 분해를 적용하는 대신, 사전 학습 단계에서부터 모델을 분해할 수는 없을까요? 저랭크 사전 학습은 모델의 매개변수 수를 크게 줄이고, 모델의 사전 학습 시간과 비용을 크게 절감할 수 있습니다.

2010년대 동안, 많은 연구자들이 저랭크 신경망을 학습시키려는 시도를 했습니다. 예를 들어, "고차원 출력 타겟을 가진 심층 신경망 학습을 위한 저랭크 행렬 분해"(Sainath 외, 2013), "심층 신경망을 위한 반직교 저랭크 행렬 분해"(Povey 외, 2018), 그리고 "저랭크 확장을 통한 컨볼루션 신경망 속도 향상"(Jaderberg 외, 2014)과 같은 연구들이 이를 보여줍니다.

저랭크 행렬 분해는 소규모 모델에서 매우 효과적임이 입증되었습니다. 예를 들어, 다양한 분해 전략(3×3 컨볼루션을 1×1 컨볼루션으로 대체)을 적용하여 **SqueezeNet**(Iandola 외, 2016)은 **AlexNet 수준의 정확도**를 ImageNet에서 달성하면서도 매개변수를 50배 줄였습니다.

최근에는 저랭크 LLM 학습에 대한 시도가 더 진행되었으며, 이에 포함된 연구로 **ReLoRA**(Lialin 외, 2023)와 **GaLore**(Zhao 외, 2024)가 있습니다. ReLoRA는 최대 13억(1.3B) 개의 매개변수를 가진 Transformer 기반 모델에서 작동하며, GaLore는 10억(1B) 개의 매개변수를 가진 모델에서 풀랭크 모델과 유사한 성능을 달성하고, 70억(7B) 개의 매개변수에서도 유망한 성능을 보여줍니다.

---

**LoRA 구성**

LoRA를 적용하려면, 어떤 가중치 행렬에 LoRA를 적용할지, 그리고 각 분해의 랭크(rank)를 결정해야 합니다. LoRA의 효율성은 LoRA가 적용되는 행렬뿐만 아니라, 모델의 구조에도 의존합니다. 서로 다른 구조는 서로 다른 가중치 행렬을 가지기 때문입니다.

LoRA는 주로 **쿼리 행렬(Wq)**, **키 행렬(Wk)**, **값 행렬(Wv)**, **출력 투영 행렬(Wo)**과 같은 Transformer 모델의 주의(attention) 모듈에 있는 네 가지 가중치 행렬에 적용됩니다.

LoRA는 일반적으로 모델 내 동일한 유형의 모든 행렬에 균일하게 적용됩니다. 예를 들어, 쿼리 행렬에 LoRA를 적용하면, 모델 내 모든 쿼리 행렬에 LoRA가 적용됩니다.

**GPT-3 175B의 파인튜닝 예시**

Hu 외(2021)는 GPT-3 175B 모델의 파인튜닝에서 학습 가능한 매개변수 예산을 **1800만(18M)**으로 설정했습니다. 이는 모델 전체 매개변수의 0.01%에 해당합니다. 이 예산은 다음을 허용합니다:

1. **랭크 8의 하나의 행렬**  
2. **랭크 4의 두 개의 행렬**  
3. **랭크 2의 네 개의 행렬**  

---

>**참고**
>
>GPT-3 175B 모델은 모델 차원이 12,288이고, 96개의 Transformer 레이어를 가지고 있습니다. LoRA를 적용할 때, 랭크 $ r = 2 $로 모든 네 개의 행렬에 적용하면, 레이어당 $ (12,288 \times 2 \times 2) \times 4 = 196,608 $개의 학습 가능한 매개변수가 생성됩니다. 이는 전체 모델에서 총 **18,874,368개의 학습 가능한 매개변수**를 생성합니다.

---

LoRA를 모든 네 개의 행렬에 대해 랭크 $ r = 2 $로 적용했을 때, WikiSQL(Zhong 외, 2017) 및 MultiNLI(Multi-Genre Natural Language Inference) 벤치마크(Williams 외, 2017)에서 최고의 성능을 보였습니다. **표 7-5**는 그 결과를 보여줍니다. 그러나 연구자들은 주의 행렬 중 두 개만 선택할 수 있다면, 쿼리(query)와 값(value) 행렬이 일반적으로 최고의 결과를 제공한다고 제안했습니다.

**표 7-5. LoRA의 학습 가능한 매개변수 예산 18M일 때의 성능 (Hu 외, 2021에서 발췌)**

| **가중치 유형** | $ W_q $ | $ W_k $ | $ W_v $ | $ W_o $ | $ W_q, W_k $ | $ W_q, W_v $ | $ W_q, W_k, W_v, W_o $ |
|-----------------|----------|----------|----------|----------|----------------|----------------|--------------------------|
| **랭크 $ r $**   | 8        | 8        | 8        | 8        | 4              | 4              | 2                        |
| **WikiSQL ($\pm 0.5\%$)** | 70.4     | 70.0     | 73.0     | 73.2     | 71.4           | **73.7**           | **73.7**                |
| **MultiNLI ($\pm 0.1\%$)** | 91.0     | 90.8     | 91.0     | 91.3     | 91.3           | 91.3           | **91.7**                |

**실험적 관찰 결과에 따르면, LoRA를 피드포워드 행렬을 포함한 더 많은 가중치 행렬에 적용하면 더 나은 결과를 가져옵니다.** 예를 들어, Databricks는 모든 피드포워드 레이어에 LoRA를 적용했을 때 가장 큰 성능 향상을 얻었다고 보고했습니다(Sooriyarachchi, 2023). Fomenko 외(2024)는 피드포워드 기반 LoRA가 주의(attention) 기반 LoRA를 보완할 수 있지만, 주의 기반 LoRA가 일반적으로 메모리 제약 내에서 더 큰 효율성을 제공한다고 언급했습니다.

LoRA의 장점 중 하나는 성능이 랭크에 따라 달라지지만, 연구 결과에 따르면 $ r = 4 $에서 $ r = 64 $ 사이의 작은 랭크가 대부분의 사용 사례에서 충분히 적합하다는 점입니다. 작은 $ r $는 LoRA 매개변수가 적다는 것을 의미하며, 이는 더 낮은 메모리 사용량으로 이어집니다.

LoRA 저자들은 놀랍게도 $ r $ 값을 증가시키는 것이 파인튜닝 성능을 향상시키지 않는다는 점을 관찰했습니다. 이 관찰은 Databricks의 보고서와 일치하며, "특정 값을 초과하여 $ r $를 증가시키는 것은 모델 출력 품질에서 뚜렷한 향상을 가져오지 않을 수 있다"고 언급했습니다(Sooriyarachchi, 2023). 일부 연구자들은 높은 $ r $가 과적합(overfitting)을 초래할 수 있다고 주장하지만, 특정 작업에서는 높은 랭크가 필요할 수 있습니다. 예를 들어, Raschka(2023)는 $ r = 256 $일 때 특정 작업에서 최고의 성능을 달성했다고 보고했습니다.

또 다른 LoRA 하이퍼파라미터는 $ \alpha $로, 이는 병합 시 $ W_{AB} $가 새로운 행렬에 얼마나 기여할지를 결정합니다. 수식은 다음과 같습니다:
$
W' = W + \frac{\alpha}{r} W_{AB}
$
실제로 $ \alpha $와 $ r $의 비율 $ \alpha:r $은 일반적으로 $ 1:8 $에서 $ 8:1 $ 사이로 설정됩니다. 하지만 최적의 비율은 작업에 따라 다릅니다. 예를 들어, $ r $가 작다면 $ \alpha $를 더 크게 설정하는 것이 좋을 수 있고, $ r $가 크다면 $ \alpha $를 더 작게 설정할 수 있습니다. 최적의 $ (r, \alpha) $ 조합을 결정하려면 실험이 필요합니다.

---

**LoRA 어댑터 서빙**

LoRA는 더 적은 메모리와 데이터로 모델을 파인튜닝할 수 있게 해줄 뿐만 아니라, 모듈식 설계 덕분에 여러 모델을 서빙하는 과정도 단순화합니다. LoRA로 파인튜닝된 모델을 서빙하는 방법은 크게 두 가지가 있습니다:

1. **LoRA 가중치 $ A $와 $ B $를 원래 모델에 병합하여 새로운 행렬 $ W' $를 생성한 후 서빙** 합니다. 이 방식은 추론 중 추가 계산이 필요하지 않으므로 **추가 지연(latency)** 이 발생하지 않습니다.

2. **LoRA 가중치 $ W $, $ A $, $ B $를 별도로 유지하며 서빙** 합니다. 이 방식은 추론 중에 $ A $와 $ B $를 $ W $에 병합하는 과정을 포함하므로 **추가 지연**이 발생합니다.

첫 번째 옵션은 LoRA 모델 하나만 서빙할 경우 더 적합합니다. 반면, 두 번째 옵션은 **다중 LoRA 모델(multi-LoRA)**을 서빙할 경우 더 적합하며, 여러 LoRA 모델이 동일한 기본(base) 모델을 공유하는 경우에 사용됩니다. **그림 7-12**는 LoRA 어댑터를 별도로 유지하면서 다중 LoRA를 서빙하는 방식을 시각화합니다.

<img src="./images/fig_07_12.png" width="800">

다중 LoRA 서빙에서 옵션 2는 지연 시간이 추가되지만, 필요한 저장 공간을 크게 줄일 수 있습니다. LoRA를 사용하여 각 고객마다 모델을 파인튜닝하는 시나리오를 생각해보겠습니다. 100명의 고객이 있다면, 동일한 기본 모델을 공유하는 100개의 파인튜닝된 모델이 생성됩니다. 옵션 1에서는 100개의 풀랭크 행렬 W'를 저장해야 하지만, 옵션 2에서는 하나의 풀랭크 행렬 W와 100개의 작은 행렬(A, B) 세트만 저장하면 됩니다.

이를 구체적으로 살펴보기 위해, 원본 행렬 W가 4096 × 4096 (16.8M 매개변수) 차원이고 LoRA의 랭크가 8이라고 가정하면, A와 B의 매개변수 수는 4096 × 8 × 2 = 65,536입니다:
- **옵션 1**: 100개의 풀랭크 행렬 $ W' $의 총 매개변수는 $ 16.8M \times 100 = 1.68B $입니다.  
- **옵션 2**: 하나의 풀랭크 행렬 $ W $와 100개의 작은 행렬(A, B)의 총 매개변수는 $ 16.8M + 65,536 \times 100 = 23.3M $입니다.

**옵션 2는 작업 간 전환도 더 빠르게 만듭니다.** 예를 들어, 고객 X의 모델을 사용 중이고 고객 Y로 전환하려면, 고객 Y의 전체 가중치 행렬을 로드하는 대신 Y의 LoRA 어댑터만 로드하면 됩니다. 이는 로딩 시간을 크게 줄일 수 있습니다. A와 B를 별도로 유지하면 추가 지연 시간이 발생하지만, 이러한 지연 시간을 최소화할 수 있는 최적화 기술이 존재합니다. 이 책의 GitHub 저장소에는 이를 수행하는 방법에 대한 설명이 포함되어 있습니다.

**Multi-LoRA 서빙은 여러 특화된 모델을 결합하는 것을 쉽게 만듭니다.** 모든 작업을 위한 하나의 강력한 모델을 사용하는 대신, 작업마다 하나의 LoRA 어댑터를 사용할 수 있습니다. 예를 들어, Apple은 동일한 30억(3B) 매개변수 기본 모델을 다양한 iPhone 기능에 맞추기 위해 여러 LoRA 어댑터를 사용했습니다(2024). 이들은 양자화 기술을 활용하여 기본 모델과 어댑터의 메모리 사용량을 더욱 줄였으며, 이를 통해 모든 작업을 디바이스에서 서빙할 수 있었습니다.

**LoRA 어댑터의 모듈성 덕분에, 어댑터는 공유 및 재사용이 가능합니다.** 공개적으로 사용할 수 있는 파인튜닝된 LoRA 어댑터가 있으며, 이는 사전 학습된 모델처럼 사용할 수 있습니다. Hugging Face나 AdapterHub와 같은 플랫폼에서 이를 찾을 수 있습니다.

---

**양자화된 LoRA**

LoRA의 빠른 발전은 다양한 LoRA 변형의 개발로 이어졌습니다. 일부 변형은 학습 가능한 매개변수의 수를 더 줄이는 것을 목표로 합니다. 그러나 **표 7-6**에서 볼 수 있듯이, LoRA 어댑터의 메모리는 모델 가중치 메모리에 비해 매우 적습니다. LoRA 매개변수의 수를 줄이면 모델의 전체 메모리 사용량은 약간만 감소합니다.

**표 7-6. 모델 가중치와 LoRA 어댑터의 메모리 요구량 비교**

| **모델 이름**         | **모델 가중치 메모리 (16비트)** | **LoRA 학습 가능한 매개변수 (r=2, 쿼리 및 키 행렬)** | **LoRA 어댑터 메모리 (16비트)** |
|------------------------|-------------------------------|-----------------------------------|-----------------------------------|
| **Llama 2 (13B)**      | 26GB                          | 3.28M                             | 6.55MB                            |
| **GPT-3 (175B)**       | 350GB                         | 18.87M                            | 37.7MB                            | 

**LoRA의 매개변수 수를 줄이려는 대신, 모델의 가중치, 활성화 값, 그리고/또는 그래디언트를 양자화(quantizing)하여 파인튜닝 중 메모리 사용량을 더 효과적으로 줄일 수 있습니다.** LoRA의 초기 유망한 양자화 버전 중 하나는 **QLoRA**(Dettmers 외, 2023)입니다. LoRA 원본 논문에서는, 파인튜닝 중에 모델의 가중치가 16비트로 저장됩니다. 반면, QLoRA는 모델의 가중치를 4비트로 저장하고, 순방향(forward) 및 역방향(backward) 계산 시 **BF16** 형식으로 다시 디양자화(dequantize)합니다.

QLoRA가 사용하는 4비트 형식은 **NF4(NormalFloat-4)**로, 사전 학습된 가중치가 보통 **중앙값이 0인 정규 분포(normal distribution)**를 따른다는 통찰에 기반하여 값을 양자화합니다. 4비트 양자화 외에도, QLoRA는 **페이지드 옵티마이저(paged optimizers)**를 사용하여 GPU 메모리가 부족할 때 데이터를 CPU와 GPU 간에 자동으로 전송합니다. 이는 특히 긴 시퀀스 길이에서 유용합니다. 이러한 기술 덕분에 **650억(65B)** 매개변수를 가진 모델을 단일 **48GB GPU**에서 파인튜닝할 수 있게 되었습니다.

저자들은 Llama 7B에서 65B에 이르는 다양한 모델을 4비트 모드로 파인튜닝했습니다. 이로 인해 생성된 모델군은 **Guanaco**라고 불리며, 공공 벤치마크와 비교 평가에서 경쟁력 있는 성능을 보여주었습니다. **표 7-7**은 이러한 모델의 Elo 점수를 보여줍니다.

**표 7-7. Guanaco 모델의 Elo 점수 (2023년 5월, GPT-4로 평가)**  
이 실험은 QLoRA(Dettmers 외, 2023)에서 수행되었습니다.

| **모델**          | **크기** | **Elo**          |
|-------------------|----------|------------------|
| **GPT-4**         | -        | 1348 ± 1         |
| **Guanaco 65B**   | 41 GB    | 1022 ± 1         |
| **Guanaco 33B**   | 21 GB    | 992 ± 1          |
| **Vicuna 13B**    | 26 GB    | 974 ± 1          |
| **ChatGPT**       | -        | 966 ± 1          |
| **Guanaco 13B**   | 10 GB    | 916 ± 1          |
| **Bard**          | -        | 902 ± 1          |
| **Guanaco 7B**    | 6 GB     | 879 ± 1          |

**QLoRA의 주요 한계는 NF4 양자화가 비용이 많이 든다는 점입니다.** QLoRA는 메모리 사용량을 줄일 수 있지만, 양자화 및 디양자화 단계에 추가 시간이 소요되기 때문에 학습 시간이 늘어날 수 있습니다.

메모리 절감 효과 덕분에, 양자화된 LoRA는 활발한 연구 분야로 자리 잡고 있습니다. QLoRA 외에도 **QA-LoRA**(Xu 외, 2023), **ModuLoRA**(Yin 외, 2023), **IR-QLoRA**(Qin 외, 2024)와 같은 다른 양자화된 LoRA 연구들이 진행되고 있습니다.

---

### **모델 병합과 다중 작업 파인튜닝**

파인튜닝이 단일 모델을 수정하여 사용자 지정 모델을 생성할 수 있게 해주는 반면, 모델 병합은 여러 모델을 결합하여 사용자 지정 모델을 생성할 수 있게 해줍니다. 모델 병합은 단순한 파인튜닝보다 더 큰 유연성을 제공합니다. 기존의 두 가지 모델을 결합하여 새로운 모델, 더욱 유용할 것으로 기대되는 모델을 만들 수 있습니다. 또한, 병합하기 전에 구성 모델 중 일부 또는 전체를 파인튜닝할 수도 있습니다.

병합된 모델을 추가로 파인튜닝하지 않아도 되지만, 파인튜닝을 통해 성능을 종종 개선할 수 있습니다. 파인튜닝 없이도 모델 병합은 GPU 없이 수행할 수 있으므로, 많은 계산 리소스에 접근할 수 없는 독립적인 모델 개발자들에게 특히 매력적입니다.

모델 병합의 목표는 구성 모델 각각을 개별적으로 사용하는 것보다 더 많은 가치를 제공하는 단일 모델을 만드는 것입니다. 이러한 부가 가치는 성능 향상을 통해 얻을 수 있습니다. 예를 들어, 동일한 작업에서 각기 다른 강점을 가진 두 모델이 있다면, 이 두 모델을 병합하여 해당 작업에서 두 모델 모두보다 더 나은 단일 모델을 만들 수 있습니다. 한 모델이 전체 질문의 처음 60%를 답할 수 있고, 다른 모델이 마지막 60%를 답할 수 있다고 가정해 보세요. 이들을 결합하면, 전체 질문의 80%를 답할 수 있을지도 모릅니다.

또한, 부가 가치는 메모리 사용량 감소에서 올 수도 있습니다. 메모리 사용량 감소는 비용 절감으로 이어질 수 있습니다. 예를 들어, 서로 다른 작업을 수행할 수 있는 두 모델이 있다면, 이 모델들을 병합하여 더 적은 매개변수로 두 작업을 모두 수행할 수 있는 단일 모델을 만들 수 있습니다. 이는 특히 어댑터 기반 모델에서 매력적입니다. 동일한 기본 모델 위에서 파인튜닝된 두 모델이 주어졌을 때, 이들의 어댑터를 단일 어댑터로 결합할 수 있습니다.

모델 병합의 중요한 사용 사례 중 하나는 **다중 작업 파인튜닝**입니다. 모델 병합 없이 여러 작업을 위해 모델을 파인튜닝하려면, 일반적으로 다음 접근 방식 중 하나를 따라야 합니다.


- **동시 파인튜닝(Simultaneous finetuning)**
    모든 작업에 대한 예제를 포함한 데이터셋을 생성하고, 이 데이터셋을 사용해 모델을 파인튜닝하여 모델이 모든 작업을 동시에 학습하도록 합니다. 하지만 여러 기술을 동시에 학습하는 것은 일반적으로 더 어렵기 때문에, 이 접근 방식은 더 많은 데이터와 더 많은 학습을 필요로 하는 경우가 많습니다.
    
- **순차 파인튜닝(Sequential finetuning)**
    모델을 각각의 작업에 대해 순차적으로 파인튜닝할 수 있습니다. 예를 들어, 작업 A에 대해 학습한 후, 작업 B에 대해 학습을 진행하는 방식입니다. 이 접근 방식은 모델이 한 번에 하나의 작업을 학습하는 것이 더 쉽다는 가정에 기반합니다. 그러나 신경망은 **파국적 망각(catastrophic forgetting)**에 취약합니다(Kirkpatrick 외, 2016). 즉, 새로운 작업에 대해 학습할 때 이전 작업을 수행하는 방법을 잊어버릴 수 있으며, 이는 이전 작업에 대한 성능이 크게 떨어지는 결과를 초래할 수 있습니다.

**모델 병합은 다중 작업 파인튜닝을 위한 또 다른 방법을 제공합니다.** 모델을 서로 다른 작업에 대해 개별적으로 파인튜닝한 뒤, 이들 모델을 병합합니다. 각 작업에 대해 개별적으로 파인튜닝하면 모델이 해당 작업을 더 잘 학습할 수 있습니다. 또한 순차적 학습이 없으므로 **파국적 망각(catastrophic forgetting)**의 위험이 줄어듭니다.

모델 병합은 휴대폰, 노트북, 자동차, 스마트워치, 창고 로봇과 같은 기기에 모델을 배포해야 할 때도 유용합니다. 기기 상에서의 배포(on-device deployment)는 제한된 메모리 용량 때문에 종종 어려움을 겪습니다. 여러 작업에 대해 각각의 모델을 기기에 배포하는 대신, 이러한 모델을 병합하여 여러 작업을 수행하면서도 메모리 사용량이 훨씬 적은 단일 모델을 만들 수 있습니다.

기기 상 배포는 데이터가 기기를 벗어나지 않아야 하는 경우(주로 **프라이버시 문제**)나, 인터넷 접속이 제한되거나 신뢰할 수 없는 경우에 필요합니다. 기기 상 배포는 또한 추론 비용을 상당히 줄일 수 있습니다. 사용자의 기기로 더 많은 연산을 전가할수록 데이터 센터에 지불해야 할 비용이 줄어듭니다.

**모델 병합은 연합 학습(federated learning)**의 한 방법으로 사용될 수 있습니다(McMahan 외, 2016). 연합 학습에서는 여러 기기가 서로 다른 데이터를 사용하여 동일한 모델을 학습합니다. 예를 들어, 모델 X를 여러 기기에 배포하면, 각 기기의 X는 해당 기기의 데이터로 개별적으로 학습을 계속할 수 있습니다. 시간이 지나면, 서로 다른 데이터로 학습된 X의 여러 복사본이 생깁니다. 이후 이러한 복사본을 병합하여 모든 구성 모델의 학습 내용을 포함하는 새로운 기본 모델을 만들 수 있습니다.

모델을 결합하여 더 나은 성능을 얻으려는 아이디어는 **모델 앙상블(ensemble) 방법**에서 시작되었습니다. Wikipedia에 따르면, 앙상블은 "여러 학습 알고리즘을 결합하여 개별 학습 알고리즘보다 더 나은 예측 성능을 얻는 것"입니다. 모델 병합이 구성 모델의 매개변수를 혼합하는 것을 포함하는 경우가 많은 반면, 앙상블은 일반적으로 구성 모델 각각을 유지하면서 **모델의 출력만 결합**합니다.

예를 들어, 앙상블에서는 하나의 쿼리에 대해 세 개의 모델을 사용해 각각 세 가지 답변을 생성한 뒤, 이 답변들을 바탕으로 다수결 방식이나 또 다른 학습 가능한 ML 모듈을 사용해 최종 답변을 생성합니다. 앙상블은 일반적으로 성능을 향상시킬 수 있지만, 요청당 여러 번의 추론 호출이 필요하므로 **추론 비용이 더 높습니다.**

**그림 7-13**은 앙상블과 모델 병합을 비교합니다. 과거 모델 앙상블이 리더보드를 지배했던 것처럼, Hugging Face의 Open LLM Leaderboard 상위권 모델 중 많은 부분이 병합된 모델입니다.  

<img src="./images/fig_07_13.png" width="800">

**많은 모델 병합 기술이 실험적인 상태이며, 커뮤니티가 기반 이론을 더 잘 이해하게 되면 구식이 될 수 있습니다.** 이러한 이유로, 개별 기술보다는 고수준의 병합 접근 방식에 초점을 맞추겠습니다.

**모델 병합 접근 방식은 구성 매개변수를 결합하는 방법에 따라 다릅니다.** 여기에서 다루는 세 가지 접근 방식은 **합산(summing)**, **레이어 스태킹(layer stacking)**, **연결(concatenation)**입니다. **그림 7-14**는 이들 접근 방식의 고수준 차이를 보여줍니다.

<img src="./images/fig_07_14.png" width="800">

**모델을 병합할 때, 이러한 접근 방식을 혼합할 수도 있습니다.** 예를 들어, 일부 레이어를 합산하고 다른 레이어를 스택(stack)할 수 있습니다. 이제 각 접근 방식을 자세히 살펴보겠습니다.

---

**합산(Summing)**

이 접근 방식은 구성 모델의 가중치 값을 서로 더하는 것을 포함합니다. 여기서는 두 가지 합산 방법을 다룹니다: **선형 결합(linear combination)**과 **구면 선형 보간(spherical linear interpolation)**. 두 모델의 매개변수가 서로 다른 스케일(예: 한 모델의 매개변수 값이 다른 모델보다 훨씬 큰 경우)에 있다면, 매개변수 값을 동일한 범위로 조정한 후 합산할 수 있습니다.

---

**선형 결합(Linear combination)**

선형 결합은 평균과 가중 평균을 모두 포함합니다. 두 모델 A와 B가 주어졌을 때, 이들의 가중 평균은 다음과 같습니다:

$
\text{Merge}(A, B) = \frac{W_AA + W_BB}{W_A + W_B}
$

**그림 7-15**는 $ W_A = W_B = 1 $일 때 두 레이어를 선형적으로 결합하는 방법을 보여줍니다.

<img src="./images/fig_07_15.png" width="800">

**선형 결합은 놀라울 정도로 단순한 방법임에도 불구하고 매우 잘 작동합니다.** 1990년대 초반(Perrone, 1993)부터 여러 모델을 선형적으로 결합하여 더 나은 모델을 만드는 아이디어가 연구되었습니다. 선형 결합은 종종 **연합 학습(federated learning)**에서도 사용됩니다(Wang 외, 2020).

**전체 모델이나 모델의 일부를 선형적으로 결합할 수 있습니다.** 예를 들어, **Model soups**(Wortsman 외, 2022)는 전체 가중치를 평균 내어 여러 파인튜닝된 모델의 정확도를 향상시킬 수 있음을 보여주었습니다. 이는 추론 시간 증가 없이 가능합니다. 하지만 모델 병합은 일반적으로 특정 구성 요소(예: 어댑터)를 결합하는 방식으로 이루어집니다.

**선형 결합은 동일한 기본(base) 모델을 바탕으로 파인튜닝된 모델의 경우 가장 효과적입니다.** 이 경우, 선형 결합은 **태스크 벡터(task vector)**의 개념으로 이해할 수 있습니다. 특정 작업에 대해 모델을 파인튜닝한 후 기본 모델에서 이를 빼면 해당 작업의 본질을 포착하는 벡터를 얻을 수 있습니다. 태스크 벡터는 **델타 매개변수(delta parameters)**라고도 합니다. LoRA를 사용해 파인튜닝했다면, LoRA 가중치에서 태스크 벡터를 추출할 수 있습니다.

태스크 벡터를 사용하면 **태스크 산술(task arithmetic)**을 수행할 수 있습니다(Ilharco 외, 2022). 예를 들어, 두 개의 태스크 벡터를 더해 태스크 기능을 결합하거나, 특정 기능을 줄이기 위해 태스크 벡터를 뺄 수 있습니다. 태스크 벡터 빼기는 사전 학습 중에 학습된 침습적 기능(예: 얼굴 인식) 또는 편향과 같은 바람직하지 않은 모델 동작을 제거하는 데 유용합니다.

결합할 구성 요소가 동일한 아키텍처와 동일한 크기를 가진 경우, 선형 결합은 간단하고 효과적입니다. 그러나 아키텍처나 크기가 다른 모델에도 사용할 수 있습니다. 예를 들어, 한 모델의 레이어가 다른 모델의 레이어보다 크다면, 두 레이어를 동일한 차원으로 투영하여 결합할 수 있습니다.

일부 연구자들은 기능적으로 관련된 매개변수가 평균화되도록 모델을 정렬한 후 결합하는 방식을 제안했습니다. 예를 들어:  
- **“Model Fusion via Optimal Transport”** (Singh and Jaggi, 2020),  
- **“Git Re-Basin: Merging Models Modulo Permutation Symmetries”** (Ainsworth 외, 2022),  
- **“Merging by Matching Models in Task Parameter Subspaces”** (Tam 외, 2023).  

매개변수를 정렬하여 결합하는 것은 타당한 방식이지만, 이를 실행하는 것은 어려울 수 있습니다. 따라서 이런 접근 방식은 일반적인 선형 결합에서는 덜 일반적입니다.

---

**구면 선형 보간(Spherical linear interpolation, SLERP)**

모델 합산의 또 다른 일반적인 방법은 SLERP로, **구면 선형 보간(spherical linear interpolation)**이라는 수학적 연산자에 기반합니다.

---

>**주의**
>
>보간(interpolation)은 알려진 값을 바탕으로 알려지지 않은 값을 추정하는 것을 의미합니다. 모델 병합의 경우, 알려지지 않은 값은 병합된 모델이고, 알려진 값은 구성 모델입니다. 선형 결합이 하나의 보간 기법이라면, SLERP도 또 다른 기법입니다.

---

SLERP의 공식은 수학적이기 때문에 여기서는 자세히 다루지 않겠습니다. 직관적으로, SLERP는 각 구성 요소(벡터)를 구의 표면상의 점으로 간주합니다. 두 벡터를 병합하려면, 먼저 두 점 간의 최단 경로를 그립니다. 이는 지구 표면상의 두 도시를 잇는 최단 경로를 그리는 것과 비슷합니다. 병합된 벡터는 이 최단 경로를 따라 위치한 점 중 하나로 설정됩니다. 이 점의 위치는 보간 계수(interpolation factor)에 따라 결정되며, 이 계수는 **0과 1 사이의 값**으로 설정할 수 있습니다. 계수가 **0.5보다 작으면**, 병합된 벡터는 첫 번째 벡터에 더 가깝게 설정됩니다. 반대로, 계수가 **0.5보다 크면**, 병합된 벡터는 두 번째 벡터에 더 가까워집니다. 계수가 **0.5**일 때는 두 벡터 사이의 정확히 중간 지점을 선택합니다. 이 중간 지점은 **그림 7-16**에서 파란 점으로 표시됩니다.

<img src="./images/fig_07_16.png" width="800">

SLERP는 수학적 연산이기 때문에 한 번에 두 벡터만 병합할 수 있습니다. 두 개 이상의 벡터를 병합하려면, SLERP를 순차적으로 적용해야 합니다. 예를 들어, A와 B를 병합한 다음, 그 결과를 C와 병합하는 방식으로 진행합니다.

---

**'중복된 작업 특화 매개변수 제거하기'**  
파인튜닝(finetuning) 과정에서 많은 모델의 매개변수가 조정됩니다. 하지만 이러한 조정의 대부분은 미미하며, 작업에 대한 모델 성능에 크게 기여하지 않습니다. 이러한 성능에 기여하지 않는 조정들은 중복(redundant)으로 간주됩니다.  

2023년 Yadav 등이 발표한 논문 **“TIES-Merging: Resolving Interference When Merging Models”**에서는 많은 작업 벡터(task vector) 매개변수를 초기화(reset)해도 성능 저하가 최소화된다는 것을 보여주었습니다. 이는 그림 7-17에 나타나 있습니다. 초기화는 파인튜닝된 매개변수를 기본(base) 모델의 원래 값으로 되돌리는 것을 의미하며, 이는 해당 작업 벡터 매개변수를 사실상 0으로 설정하는 것과 동일합니다. (작업 벡터는 파인튜닝된 모델에서 기본 모델을 뺀 값으로 얻을 수 있음을 기억하세요.)

<img src="./images/fig_07_17.png" width="800">

이러한 중복 매개변수는 하나의 모델에서는 해가 되지 않을 수 있지만, 병합된 모델에서는 해로 작용할 가능성이 있습니다. TIES(Yadav et al., 2023)와 DARE(Yu et al., 2023)와 같은 병합 기술은 병합 전에 작업 벡터에서 중복 매개변수를 먼저 제거합니다. 이러한 접근법이 최종 병합 모델의 품질을 크게 향상시킬 수 있다는 것이 두 논문에서 입증되었습니다. 병합해야 할 모델이 많을수록 중복 매개변수를 제거하는 것이 더욱 중요해지는데, 이는 하나의 작업에서 발생한 중복 매개변수가 다른 작업에 간섭할 기회가 많아지기 때문입니다.

---

**레이어 스태킹(Layer Stacking)**  
이 접근법에서는 하나 이상의 모델에서 서로 다른 레이어를 가져와 차례로 쌓습니다. 예를 들어, 모델 1의 첫 번째 레이어와 모델 2의 두 번째 레이어를 사용할 수 있습니다. 이 방법은 **패스스루(passthrough)** 또는 **프랑켄머징(frankenmerging)**이라고도 합니다. 이 방식으로 독특한 아키텍처와 다양한 매개변수 수를 가진 모델을 생성할 수 있습니다. 하지만, 레이어 스태킹으로 병합된 모델은 일반적으로 좋은 성능을 위해 추가적인 파인튜닝이 필요합니다.

프랑켄머징의 초기 성공 사례 중 하나는 **Goliath-120B**(alpindale, 2023)입니다. 이 모델은 파인튜닝된 두 Llama 2-70B 모델인 **Xwin**과 **Euryale**에서 각각 80개 레이어 중 72개를 가져와 병합한 것입니다.

레이어 스태킹은 **혼합 전문가(Mixture-of-Experts, MoE)** 모델을 훈련하는 데에도 사용될 수 있습니다. 이는 **“희소 업사이클링: 밀집 체크포인트에서 혼합 전문가 훈련하기”(Komatsuzaki et al., 2022)**에서 소개되었습니다. MoE를 처음부터 훈련하는 대신, 사전 훈련된 모델을 사용해 특정 레이어나 모듈의 복사본을 여러 개 만듭니다. 이후 라우터(router)를 추가하여 각 입력을 가장 적합한 복사본으로 보내고, 병합된 모델을 추가로 훈련해 성능을 개선합니다. **그림 7-18**은 이 과정을 시각적으로 설명합니다.

<img src="./images/fig_07_18.png" width="800">

Komatsuzaki 등은 레이어 스태킹이 처음부터 MoE 모델을 훈련하는 것보다 우수한 모델을 생성할 수 있음을 보여주었습니다. 이 접근법을 사용해 Together AI는 6개의 약한 오픈소스 모델을 혼합하여 **Mixture-of-Agents**를 생성했으며, 이는 일부 벤치마크에서 OpenAI의 **GPT-4**와 비슷한 성능을 달성했습니다(Wang et al., 2024).

**레이어 스태킹의 흥미로운 활용 사례는 모델 업스케일링(Model Upscaling)**입니다. 모델 업스케일링은 더 적은 자원으로 더 큰 모델을 만드는 방법을 연구하는 것입니다. 가령, 기존에 팀에서 40GB GPU에 맞게 훈련된 모델을 사용하고 있었다고 가정해봅시다. 하지만 80GB GPU를 가진 새 장비를 얻었고, 이를 통해 더 큰 모델을 제공할 수 있게 되었습니다. 이런 경우, 처음부터 새 모델을 훈련하는 대신, 기존 모델을 활용해 레이어 스태킹으로 더 큰 모델을 만들 수 있습니다.

레이어 업스케일링의 한 가지 접근법은 **깊이 방향 스케일링(Depthwise Scaling)**입니다. **Kim et al.(2023)**은 이 기술을 사용하여 7B(70억 매개변수) 모델에서 **SOLAR 10.7B**를 생성했습니다. 절차는 다음과 같습니다:

1. 원래 사전 훈련된 모델의 복사본을 생성합니다.

2. 특정 레이어를 합산(두 레이어를 합쳐 하나의 레이어로 만듦)하고 나머지 레이어를 스태킹하여 두 복사본을 병합합니다. 합산할 레이어는 목표 모델 크기에 맞도록 신중히 선택됩니다. **SOLAR 10.7B**의 경우, 16개의 레이어가 합산되어 최종적으로 32 × 2 - 16 = 48개의 레이어를 가진 모델이 만들어졌습니다.

3. 이 업스케일링된 모델을 목표 성능을 달성하도록 추가 훈련합니다.

**그림 7-19**는 이 과정을 시각적으로 설명합니다.

<img src="./images/fig_07_19.png" width="800">

---

**병합(Concatenation)**  

구성 요소 모델의 매개변수를 다양한 방식으로 더하는 대신, 이를 병합(concatenate)할 수도 있습니다. 병합된 구성 요소의 매개변수 수는 모든 구성 요소의 매개변수 수를 합한 값이 됩니다. 예를 들어, 순위가 각각 $r_1$과 $r_2$인 두 LoRA 어댑터를 병합하면, 병합된 어댑터의 순위는 $r_1 + r_2$가 됩니다(그림 7-20 참조).

**병합은 추천되지 않습니다.** 이는 개별 모델을 따로 제공하는 것에 비해 메모리 사용량을 줄이지 못하기 때문입니다. 병합이 더 나은 성능을 제공할 수도 있지만, 추가 매개변수로 인해 얻을 수 있는 성능 향상이 그만한 가치가 없을 수 있습니다.

<img src="./images/fig_07_20.png" width="800">

---


### **파인튜닝 전략(Finetuning Tactics)**  
이 장에서는 여러 파인튜닝 접근법, 해결할 수 있는 문제, 그리고 그 작동 방식에 대해 논의했습니다. 이번 마지막 섹션에서는 더 실질적인 파인튜닝 전략에 초점을 맞춥니다.

---

**파인튜닝 프레임워크와 기본 모델(Finetuning frameworks and base models)**  

파인튜닝과 관련된 여러 사항들(파인튜닝 여부 결정, 데이터 수집, 파인튜닝된 모델 유지 관리 등)은 어려울 수 있지만, 실제 파인튜닝 과정은 상대적으로 간단합니다. 세 가지를 선택해야 합니다: 기본 모델(base model), 파인튜닝 방법, 그리고 파인튜닝에 사용할 프레임워크입니다.

---

**기본 모델(Base models)**  

**4장(Chapter 4)**에서는 프롬프트 기반 방법과 파인튜닝 모두에 적용할 수 있는 모델 선택 기준을 다뤘습니다. 논의된 기준에는 모델 크기, 라이선스, 벤치마크 성능 등이 포함됩니다. AI 프로젝트 초기에, 작업의 실행 가능성을 탐색하는 단계에서는 사용할 수 있는 가장 강력한 모델로 시작하는 것이 유용합니다. 이 모델이 좋은 결과를 내지 못한다면, 더 약한 모델은 더 나쁜 결과를 낼 가능성이 높습니다. 가장 강력한 모델이 요구를 충족한다면, 초기 모델을 비교 기준으로 사용하여 더 약한 모델도 탐색할 수 있습니다.

파인튜닝을 위해 시작 모델은 프로젝트에 따라 다릅니다. **OpenAI의 파인튜닝 모범 사례 문서**에서는 두 가지 개발 경로를 예로 들고 있습니다: **진행 경로(progression path)**와 **지식 증류 경로(distillation path)**입니다.

**진행 경로(progression path)**는 다음과 같이 진행됩니다:

1. 가장 저렴하고 빠른 모델을 사용하여 코드가 예상대로 작동하는지 확인하면서 파인튜닝 코드를 테스트합니다.

2. 중간 수준 모델로 파인튜닝하여 데이터를 테스트합니다. 만약 데이터가 증가해도 훈련 손실이 줄어들지 않는다면, 문제가 있을 수 있습니다.

3. 최고의 성능을 내기 위해 가장 강력한 모델로 몇 가지 추가 실험을 진행합니다.

4. 좋은 결과를 얻은 후, 모든 모델로 훈련을 진행하여 가격/성능 지표를 설정하고 사용 사례에 가장 적합한 모델을 선택합니다.

**지식 증류 경로(Distillation Path)**는 다음과 같이 진행될 수 있습니다:

1. **작은 데이터셋과 가장 강력한 모델로 시작합니다.** 사용할 수 있는 가장 강력한 모델을 선택하고, 이 작은 데이터셋을 사용해 가능한 최상의 모델을 훈련합니다. 기본 모델이 이미 강력하기 때문에, 좋은 성능을 얻는 데 적은 데이터만 필요합니다.

2. **이 파인튜닝된 모델을 사용해 더 많은 훈련 데이터를 생성합니다.**

3. **새로 생성된 데이터를 사용해 더 저렴한 모델을 훈련합니다.**

파인튜닝은 일반적으로 프롬프트 엔지니어링 실험 이후에 진행되기 때문에, 파인튜닝을 시작할 시점에는 이상적으로 다양한 모델의 동작에 대해 상당히 잘 이해하고 있어야 합니다. 이러한 이해를 바탕으로 파인튜닝 개발 경로를 계획해야 합니다.

--- 

다음은 이미지의 내용을 번역한 결과입니다:

---

**파인튜닝 방법(Finetuning methods)**  

LoRA와 같은 어댑터 기술은 비용 효율적이지만 일반적으로 풀 파인튜닝(full finetuning)만큼의 성능을 제공하지는 못합니다. 파인튜닝을 처음 시작할 때는 LoRA와 같은 방법을 시도하고, 나중에 풀 파인튜닝을 시도하는 것이 좋습니다.

파인튜닝 방법은 데이터 양에도 의존합니다. 기본 모델과 작업에 따라 풀 파인튜닝은 종종 수천에서 수만 개의 예제가 필요하고 비용이 많이 들 수 있습니다. 하지만 **PEFT** 방법은 훨씬 적은 데이터셋(예: 수백 개의 예제)으로도 좋은 성능을 보일 수 있습니다. 작은 데이터셋만 있다면, 풀 파인튜닝이 LoRA보다 나은 결과를 보장하지 않을 수도 있습니다.

파인튜닝된 모델이 몇 개나 필요한지, 그리고 이를 어떻게 제공할지를 고려해야 합니다. LoRA와 같은 어댑터 기반 방법은 동일한 기본 모델을 공유하는 여러 모델을 효율적으로 제공할 수 있도록 해줍니다. LoRA를 사용하면 하나의 기본 모델만 제공하면 되지만, 풀 파인튜닝을 사용하면 전체 모델을 각각 제공해야 합니다.

---

**파인튜닝 프레임워크(Finetuning frameworks)**  

가장 쉬운 파인튜닝 방법은 파인튜닝 API를 사용하는 것입니다. 여기서 데이터셋을 업로드하고, 기본 모델을 선택하고, 파인튜닝된 모델을 받을 수 있습니다. 모델 추론 API처럼, 파인튜닝 API는 모델 제공업체, 클라우드 서비스 제공업체, 또는 서드파티 제공업체에서 제공될 수 있습니다. 하지만 이 방법은 API가 지원하는 모델로만 제한되며, 모든 세부 조정 기능을 제공하지 않을 수 있다는 한계가 있습니다. 빠르고 간단한 방법을 원하는 사람에게는 적합하지만, 더 많은 커스터마이징을 원하는 사람에게는 불편할 수 있습니다.

**LLaMA-Factory, unsloth, PEFT, Axolotl, LitGPT**와 같은 다양한 파인튜닝 프레임워크를 사용해 파인튜닝할 수도 있습니다. 이러한 프레임워크는 특히 어댑터 기반 기술을 지원하는 다양한 파인튜닝 방법을 제공합니다. 풀 파인튜닝을 원할 경우, GitHub에서 제공되는 기본 모델의 오픈 소스 훈련 코드를 클론하여 자체 데이터를 사용해 실행할 수 있습니다. **Llama Police**는 더 종합적이고 최신 파인튜닝 프레임워크와 모델 저장소를 제공합니다.

자체 파인튜닝을 수행하면 더 많은 유연성을 제공하지만, 필요한 컴퓨팅 자원을 마련해야 합니다. 어댑터 기반 기술만 사용한다면 중간급 GPU로도 대부분의 모델에 충분할 수 있습니다. 더 많은 컴퓨팅이 필요하다면, 클라우드 제공업체와 원활하게 통합되는 프레임워크를 선택할 수 있습니다.  

여러 대의 머신을 사용해 모델을 파인튜닝하려면 **DeepSpeed, PyTorch Distributed, ColossalAI**와 같은 분산 훈련을 지원하는 프레임워크가 필요합니다.

---

**파인튜닝 하이퍼파라미터(Finetuning hyperparameters)**  
기본 모델과 파인튜닝 방법에 따라 조정할 수 있는 많은 하이퍼파라미터가 있습니다. 사용 사례에 맞는 특정 하이퍼파라미터를 확인하려면, 사용하는 기본 모델 또는 파인튜닝 프레임워크의 문서를 참조하세요. 여기에서는 자주 조정되는 몇 가지 중요한 하이퍼파라미터를 다룹니다.

---

**학습률(Learning rate)**  

학습률은 각 학습 단계에서 모델의 매개변수가 얼마나 빨리 변경될지를 결정합니다. 학습을 목표에 도달하는 길이라고 생각한다면, 학습률은 걸음의 크기라고 할 수 있습니다. 걸음 크기가 너무 작으면 목표에 도달하는 데 너무 오래 걸릴 수 있고, 너무 크면 목표를 초과하거나 모델이 수렴하지 않을 수 있습니다.

보편적인 학습률은 존재하지 않습니다. 일반적으로 \(1e-7\)에서 \(1e-3\) 사이의 학습률로 실험해본 후 적합한 값을 찾습니다. 한 가지 일반적인 방법은 사전 훈련 단계의 학습률 끝값을 가져와 \(0.1\)에서 \(1\) 사이의 일정한 값으로 곱하는 것입니다.

학습 손실 곡선을 보면 학습률에 대한 힌트를 얻을 수 있습니다. 손실 곡선이 많이 변동한다면 학습률이 너무 클 가능성이 높습니다. 손실 곡선이 안정적이지만 감소 속도가 느리다면, 학습률을 높여야 합니다. 손실 곡선이 안정적일 때까지 학습률을 올려 보세요.

훈련 과정에서 학습률을 변화시킬 수도 있습니다. 초반에는 큰 학습률을 사용하고, 후반에는 작은 학습률을 사용하는 방식입니다. 이렇게 훈련 과정에서 학습률을 점진적으로 조정하는 알고리즘을 **학습률 스케줄링(learning rate schedules)**이라고 합니다.

---

**배치 크기(Batch size)**  

배치 크기는 모델이 각 단계에서 가중치를 업데이트하기 위해 학습하는 예제의 개수를 결정합니다. 배치 크기가 너무 작아, 예를 들어 8개 미만인 경우, 훈련이 불안정해질 수 있습니다. **더 큰 배치 크기**는 서로 다른 예제에서 나온 신호를 집계하는 데 도움이 되며, 이로 인해 더 안정적이고 신뢰할 수 있는 업데이트가 이루어집니다.

**일반적으로, 배치 크기가 클수록 모델이 학습 예제를 더 빠르게 처리할 수 있습니다.** 하지만 배치 크기가 클수록 모델 실행에 필요한 메모리가 증가합니다. 따라서 배치 크기는 사용하는 하드웨어에 의해 제한됩니다.

이것이 **비용과 효율성 간의 균형(cost versus efficiency trade-off)**이 나타나는 지점입니다. 더 비싼 컴퓨팅 자원은 더 빠른 파인튜닝을 가능하게 합니다.

현재로서는 파인튜닝에서 컴퓨팅이 여전히 병목현상으로 작용합니다. 모델은 너무 크고 메모리는 제한적이어서 작은 배치 크기만 사용할 수 있는 경우가 많습니다. 이로 인해 모델 가중치 업데이트가 불안정해질 수 있습니다. 이를 해결하기 위해, 각 배치 이후에 모델 가중치를 업데이트하는 대신, **여러 배치에 걸쳐 그래디언트를 누적한 후 충분히 신뢰할 수 있는 그래디언트가 축적되면 가중치를 업데이트하는** 방법을 사용할 수 있습니다. 이 기술을 **그래디언트 누적(gradient accumulation)**이라고 합니다.

컴퓨팅 비용이 가장 중요한 요소가 아닐 때는 다양한 배치 크기를 실험하여 최적의 모델 성능을 제공하는 배치 크기를 찾을 수 있습니다.

---

**에포크 수(Number of epochs)**  

에포크(epoch)는 학습 데이터 전체를 한 번 통과하는 것을 의미합니다. 에포크 수는 각 학습 예제가 몇 번 훈련되는지를 결정합니다.

작은 데이터셋은 큰 데이터셋보다 더 많은 에포크가 필요할 수 있습니다. 예제를 수백만 개 포함한 데이터셋의 경우 1~2 에포크가 충분할 수 있지만, 예제가 수천 개인 데이터셋은 4~10 에포크 후에도 여전히 성능 향상을 보일 수 있습니다.

**훈련 손실(training loss)과 검증 손실(validation loss)**의 차이를 통해 에포크 수에 대한 힌트를 얻을 수 있습니다. 두 손실이 계속해서 안정적으로 감소한다면, 모델은 더 많은 에포크(및 데이터)에서 이점을 얻을 수 있습니다. 훈련 손실이 계속 감소하지만 검증 손실이 증가한다면, 모델이 학습 데이터에 과적합(overfitting)되고 있을 가능성이 있으므로 에포크 수를 줄여야 할 수 있습니다.

---

**프롬프트 손실 가중치(Prompt loss weight)**  

지시형 파인튜닝(instruction finetuning)의 경우, 각 예제는 프롬프트(prompt)와 응답(response)으로 구성되며, 이 둘 모두 모델의 손실에 기여할 수 있습니다. 하지만 추론(inference) 중에는 보통 사용자가 프롬프트만 제공하며, 모델은 응답만 생성하면 됩니다. 따라서 훈련 중에는 **응답 토큰(response tokens)**이 프롬프트 토큰(prompt tokens)보다 모델 손실에 더 크게 기여해야 합니다.

**프롬프트 손실 가중치(prompt model weight)**는 프롬프트가 응답에 비해 손실에 얼마나 기여해야 하는지를 결정합니다. 이 가중치가 100%라면, 프롬프트가 손실의 전부를 차지합니다. 반대로 0%라면, 모델은 응답만 학습합니다. 일반적으로 이 가중치는 기본값으로 10%로 설정되며, 이는 모델이 프롬프트에서 일부 학습하지만 대부분은 응답에서 학습하도록 한다는 것을 의미합니다.

---

### **요약(Summary)**  

평가와 관련된 장을 제외하면, **파인튜닝(finetuning)**은 작성하기 가장 어려운 장 중 하나였습니다. 이 장은 기존 개념(전이 학습)과 새로운 개념(PEFT), 기본적인 개념(저순위 행렬 분해)과 실험적인 개념(모델 병합), 수학적 개념(메모리 계산)과 전략적 개념(하이퍼파라미터 튜닝) 등 다양한 주제를 다뤘습니다. 이처럼 서로 다른 측면을 하나의 일관된 구조로 정리하면서도 접근 가능하게 유지하는 것은 쉽지 않았습니다.

파인튜닝 과정 자체는 어렵지 않습니다. 많은 파인튜닝 프레임워크가 훈련 과정을 처리해주며, 합리적인 기본 하이퍼파라미터를 제안해 주기도 합니다.

그러나 파인튜닝을 둘러싼 맥락은 복잡합니다. 이는 **모델을 파인튜닝해야 하는지 여부**에서 시작됩니다. 이 장은 파인튜닝을 해야 하는 이유와 하지 않아야 하는 이유를 다루었으며, 많은 사람들이 자주 묻는 질문인 **"언제 파인튜닝을 해야 하고, 언제 RAG(검색 기반 생성)를 해야 하는가?"**에 대해서도 논의했습니다.

초기에는 파인튜닝이 사전 훈련(pre-training)과 비슷했습니다. 두 과정 모두 모델의 모든 가중치를 업데이트했기 때문입니다. 그러나 모델 크기가 증가함에 따라 **풀 파인튜닝(full finetuning)**은 대부분의 사용자에게 실용적이지 않게 되었습니다. 파인튜닝 중 업데이트할 매개변수가 많을수록 더 많은 메모리가 필요하기 때문입니다. 대부분의 사용자는 **기초 모델(foundation models)**에 대해 풀 파인튜닝을 수행할 충분한 자원(하드웨어, 시간, 데이터)을 보유하고 있지 않습니다.

여러 파인튜닝 기술은 동일한 목표를 가지고 개발되었습니다. 즉, **최소한의 메모리 사용량으로 강력한 성능을 달성하는 것**입니다. 예를 들어, **PEFT**는 훈련 가능한 매개변수의 수를 줄임으로써 파인튜닝의 메모리 요구 사항을 줄여줍니다. 반면, **양자화 훈련(quantized training)**은 각 값을 표현하는 데 필요한 비트 수를 줄여 이러한 메모리 병목 현상을 완화합니다.

이 장은 PEFT 개요를 제공한 후 **LoRA**로 초점을 좁혔습니다. LoRA가 작동하는 이유와 방식, 그리고 LoRA가 사용자들 사이에서 인기가 많은 이유를 다루었습니다. LoRA는 **매개변수 효율성(parameter-efficient)** 및 **데이터 효율성(data-efficient)**뿐만 아니라, **모듈화(modularity)**되어 여러 LoRA 모델을 제공하고 병합하기 훨씬 더 쉽게 만들어줍니다.

파인튜닝된 모델을 결합하는 아이디어는 **모델 병합(model merging)**으로 이어졌습니다. 그 목표는 여러 모델을 하나로 병합하여 개별 모델보다 더 잘 작동하도록 만드는 것입니다. 이 장에서는 **온디바이스 배포(on-device deployment)**부터 **모델 업스케일링(model upscaling)**까지, 모델 병합의 다양한 사용 사례와 일반적인 접근 방식을 논의했습니다.

제가 사용자들로부터 자주 듣는 의견 중 하나는, **파인튜닝 자체는 쉽지만, 파인튜닝에 필요한 데이터를 얻는 것이 어렵다**는 것입니다. 특히 **고품질로 주석이 달린 데이터**를 얻는 것이 어렵습니다. 다음 장에서는 이러한 과제에 대해 다룰 것입니다.

---

1. 일부 사람들은 이 현상을 **정렬 세금(Alignment tax)**이라고 부릅니다(Bai et al., 2020). 하지만 이 용어는 인간 선호 정렬(human preference alignment)에 대한 패널티와 혼동될 수 있습니다.

2. 많은 기업들은 "충분히 좋다"고 여기는 기술을 변경하는 것을 꺼립니다. 만약 모든 기업이 더 최적화된 솔루션을 신속히 채택했다면, 팩스기는 이미 오래전에 사라졌을 것입니다.

3. 엔지니어들이 파인튜닝이 반드시 필요하지 않다는 것을 알면서도 새로운 기술을 배우고 싶어서 이를 고집하는 사례를 몇 번 본 적이 있습니다. 저도 새로운 기술을 배우는 것을 좋아하는 엔지니어로서 이런 태도를 이해합니다. 하지만 리더십 위치에 있다면, 파인튜닝이 정말 필요한지 또는 원하는 것인지를 구별하는 것이 중요합니다.

4. **0314**는 이 GPT-4 버전이 출시된 날짜, 즉 2024년 3월 14일을 나타냅니다. 특정 날짜 표시는 버전별 성능 차이가 상당하기 때문에 중요합니다.

5. Llama 3.1 논문의 저자들(Dubey et al., 2024)과 같은 일부 사람들은, 후처리 학습(post-training)은 모델이 **"스스로 아는 것을 아는 것(know what it knows)"**에 맞춰야 한다는 원칙을 따릅니다.

6. 역전파(backpropagation) 외에, 유망한 신경망 훈련 접근법 중 하나는 **진화적 전략(evolutionary strategy)**입니다. Maheswaranathan et al.이 설명한 예는 대체 그래디언트(surrogate gradients)를 사용한 무작위 탐색(random search)을 결합하여, 실제 그래디언트를 사용하는 대신 모델 가중치를 업데이트합니다. 또 다른 흥미로운 접근법은 **직접 피드백 정렬(Direct Feedback Alignment)**입니다(Arild Nøkland, 2016).

7. 매개변수가 훈련 가능하지 않다면, 해당 매개변수를 업데이트할 필요가 없으므로 그래디언트를 계산할 필요도 없습니다.

8. 일부 사람들은 **"RuntimeError: CUDA out of memory"** 오류를 겪기 전까지는 진정한 AI를 하지 않는다고 말할지도 모릅니다.

9. 추론 메모리 계산에 대해 더 알고 싶다면, Carol Chen의 블로그 **“Transformer Inference Arithmetic”** (2022년 3월)을 참조하세요.

10. 훈련 메모리 계산에 대해 더 알고 싶다면, EleutherAI의 **“Transformer Math 101”** (Anthony et al., 2023년 4월)을 참조하세요.

11. Google은 **BFloat16**을 **"Cloud TPU에서 높은 성능을 달성하기 위한 비결"**로 소개했습니다.

12. 정수 포맷(Integer formats)은 **고정 소수점 포맷(fixed point formats)**이라고도 불립니다.

13. 범위 비트(range bits)는 **지수(exponents)**라고 불리며, 정밀도 비트(precision bits)는 **유효숫자(significands)**라고 불립니다.

다음은 캡처 이미지의 내용을 번역한 결과입니다:

---

14. 일반적으로 형식 이름의 숫자는 몇 비트를 차지하는지를 나타내지만, TF32는 실제로 32비트가 아니라 19비트입니다. 이는 FP32와의 기능적 호환성을 제안하기 위해 고안된 것으로 보입니다. 하지만 솔직히 왜 TF32라고 불리는지, 그리고 왜 TF19가 아닌지가 저를 계속 궁금하게 만듭니다. NVIDIA의 전 동료가 이상한 형식(19비트)의 이름을 "TF32"로 정한 것이 더 친숙하게 보이도록 하려는 시도였을 거라는 추측을 내놓았습니다.

15. **FP16**과 **BF16**의 혼란은 **Llama 3.1**, **Sex and Threads** 논의에서 계속되었습니다. 참고 자료: [1](링크), [2](링크), [3](링크), [4](링크); **llama.cpp**의 [**BF16과 FP16 벤치마크**](링크) 및 **Raschka의 글**(링크).

16. 수치 형식을 설계하는 것은 매력적인 분야입니다. 시스템의 품질을 훼손하지 않으면서 저정밀도 형식을 만드는 능력은 해당 시스템을 더 저렴하고 빠르게 만들며 새로운 사용 사례를 가능하게 할 수 있습니다.

17. Transformer 기반 모델의 메모리 사용량에 가장 크게 기여하는 요소 중 하나는 **KV 캐시**입니다. 이는 **9장**에서 논의됩니다.

18. IEEE 표준 원칙을 따르는 가장 작은 가능 부동 소수점(float)은 4비트입니다.

19. XnorNet 논문 저자들은 Xnor.ai라는 신생 기업을 설립했으며, 이는 모델 압축에 중점을 둔 스타트업이었습니다. **2020년 초**, 이 스타트업은 Apple에 약 **2억 달러**에 인수되었습니다.

20. 훈련 중에, 모델의 가중치는 여러 단계에 걸쳐 업데이트됩니다. 작은 반올림 오류들이 훈련 과정에서 누적되어, 원하는 성능을 얻기 어렵게 만듭니다. 게다가 손실 값은 정밀한 계산을 필요로 합니다. 손실 값의 작은 변화는 모델 매개변수를 잘못된 방향으로 업데이트하게 할 수 있습니다.

21. 개인적인 일화로, 제 팀의 많은 작업이 NVIDIA에서 혼합 정밀도 훈련에 집중되어 있었습니다. 자세한 내용은 **“NLP 및 음성 인식을 위한 혼합 정밀도 훈련(OpenSeq2Seq)”**(Huyen et al., NVIDIA Developer Technical Blog, October 2018)를 참조하세요.

22. 부분 파인튜닝(partial finetuning)에서는 출력 레이어와 가장 가까운 레이어들을 조정하는 것이 일반적입니다. 이러한 레이어는 일반적인 기능보다 작업 특화된 기능을 캡처할 가능성이 높기 때문입니다.

23. 지금까지 LoRA, PEFT와 같은 기술들의 차이를 즉석에서 명확히 설명할 수 있는 사람을 만나본 적이 없습니다.

24. LoRA를 효과적으로 사용하려면 모델의 아키텍처를 이해해야 합니다. **2장**은 Transformer 기반 모델 구성의 정확한 무게 구성을 논의합니다.

25. Fireworks와 같은 일부 파인튜닝 프레임워크는 최대 **32** 순위(rank)의 LoRA를 허용합니다. 그러나 이 제한은 성능 때문이 아니라 하드웨어의 메모리 제약 때문인 경우가 많습니다.

26. 이 어댑터들을 찾으려면 **“adapter”**, **“peft”**, **“LoRA”**를 검색하세요.

27. QLoRA는 양자화된 LoRA 작업의 유일한 예가 아닙니다. 많은 연구실에서 이를 공개적으로 논의하지 않고 작업을 진행해왔습니다.

28. 제 책 **“Designing Machine Learning Systems”**에는 **“클라우드 및 엣지에서의 머신러닝”** 섹션이 포함되어 있습니다.

29. 앙상블 방법에 대한 자세한 내용은 제 책 **“Designing Machine Learning Systems”**에서 확인할 수 있습니다.

30. 평균화는 가중치뿐만 아니라 임베딩에도 사용할 수 있습니다. 예를 들어, 문장을 주어진 단어 임베딩으로 변환하고, 그 임베딩의 평균을 취하면 문장 임베딩을 생성할 수 있습니다.

31. 가정은 파인튜닝 중 가장 큰 변화가 이루어진 매개변수가 모델의 성능에 가장 중요한 역할을 한다는 것입니다.

32. TIES는 **“Trim, Elect Sign, and merge”**의 약어이며, DARE는 **“Drop And REscale”**의 약어입니다.

33. 작업 벡터가 제거될수록 더 희박해지지만, 파인튜닝된 모델 자체는 그렇지 않습니다.

34. 대학 시절, 저는 모델을 하룻밤 동안 학습시키는 실수를 저질렀고, 비어 있는 폴더에서 체크포인트를 저장하도록 지정하여 8시간의 결과가 모두 사라졌습니다.

35. 작은 배치 크기가 불안정한 훈련으로 이어지는 이유에 대한 구체적인 설명을 찾지 못했습니다. 이와 관련된 자료가 있다면 꼭 보내주세요.

36. 그래디언트 누적은 2016년 **“부분 그래디언트 교환을 통한 분산 심층 학습”(Watcharapichat et al.)** 논문에서 처음 언급되었습니다.